<a href="https://colab.research.google.com/github/MacKumachin/Clifford-FBSM-SignalControl/blob/main/GCTAD_SOTA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# === リセット一発セル：u=0 Baseline 生成 → CGTADと比較（Colab） ===
# 1) 依存とツール
#!pip -q install -U numpy scipy nibabel dipy statsmodels pandas scikit-image
!apt-get -qq update && apt-get -qq install -y dcm2niix >/dev/null

import os, re, glob, json, numpy as np, matplotlib.pyplot as plt
from nibabel.streamlines import load as sl_load
from dipy.io.image import load_nifti
from dipy.core.gradients import gradient_table
from dipy.segment.mask import median_otsu
from dipy.reconst.csdeconv import auto_response_ssst, ConstrainedSphericalDeconvModel
from dipy.direction import peaks_from_model
from dipy.data import default_sphere
from dipy.tracking.stopping_criterion import BinaryStoppingCriterion
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.streamline import Streamlines
from dipy.io.streamline import save_trk

# 2) 設定（ここだけ編集）：あなたの CGTAD .trk の実パス
CGTAD_TRK = "/content/drive/MyDrive/CGTAD/B_streamlines.trk"  # ←必要に応じて修正

# 3) CGTAD が見つからなければ自動探索
if not os.path.exists(CGTAD_TRK):
    cands = glob.glob("/content/**/*.trk", recursive=True)
    cands = [p for p in cands if re.search(r"(B_stream|cgtad|opt)", os.path.basename(p), re.I)]
    if not cands:
        raise SystemExit("❌ CGTAD .trk が見つかりません。CGTAD_TRK を実パスに直してください。")
    CGTAD_TRK = sorted(cands)[0]
print("[CGTAD]", CGTAD_TRK)

# 4) CGTAD の種点（最初の頂点）を seeds に採用、ステップ長も推定
obj = sl_load(CGTAD_TRK)
streams = [np.asarray(s, dtype=np.float64) for s in obj.streamlines if len(s)>=2]
seeds_world = [s[0] for s in streams]
# ステップ長の中央値（mm）
def step_median_mm(s):
    d = np.diff(s, axis=0);
    return float(np.median(np.linalg.norm(d, axis=1))) if d.size else np.nan
STEP_MM = float(np.nanmedian([step_median_mm(s) for s in streams]))
if not np.isfinite(STEP_MM): STEP_MM = 0.5
print(f"[seeds] {len(seeds_world)}  [estimated step] {STEP_MM:.3f} mm")

# 5) DWI の取得：既存NIfTIを探す → 無ければ Drive から DICOM候補→自動変換
DWI_PATH=BVAL_PATH=BVECS_PATH=None
# 5-1) 既存のNIfTI三点セットを /content と Drive から検索
roots = ["/content", "/content/drive/MyDrive", "/content/drive/Shareddrives"]
def find_files(patterns):
    out=[]
    for r in roots:
        for pat in patterns:
            out += glob.glob(os.path.join(r, "**", pat), recursive=True)
    return sorted(set(out))
nii  = find_files(["*dwi*.nii*", "*data.nii*"])
bval = find_files(["*.bval", "*bval*"])
bvec = find_files(["*.bvec", "*bvec*"])
if nii and bval and bvec:
    DWI_PATH, BVAL_PATH, BVECS_PATH = nii[0], bval[0], bvec[0]

# 5-2) 無ければ Drive をマウントして DICOM を変換
if not (DWI_PATH and BVAL_PATH and BVECS_PATH):
    try:
        from google.colab import drive
        if not os.path.ismount("/content/drive"):
            drive.mount("/content/drive")
    except Exception:
        pass
    # DICOM候補（枚数が多いフォルダ）
    cands=[]
    for root in roots[1:]:
        for dp, dn, fn in os.walk(root):
            n=0
            for f in fn:
                fl=f.lower()
                if fl.endswith((".dcm",".ima",".dicom")) or bool(re.fullmatch(r"\d{5,}", f)):
                    n+=1
            if n>=30: cands.append((n,dp))
    cands.sort(reverse=True)
    if cands:
        DICOM_DIR=cands[0][1]
        print("[DICOM candidate]", DICOM_DIR, f"(files≈{cands[0][0]})")
        os.makedirs("/content/dwi_out", exist_ok=True)
        # 変換
        get_ipython().system('dcm2niix -ba y -z y -o /content/dwi_out -f dwi "{DICOM_DIR}"')
        nii  = sorted(glob.glob("/content/dwi_out/*.nii*"))
        bval = sorted(glob.glob("/content/dwi_out/*.bval"))
        bvec = sorted(glob.glob("/content/dwi_out/*.bvec"))
        if nii and bval and bvec:
            DWI_PATH, BVAL_PATH, BVECS_PATH = nii[0], bval[0], bvec[0]

if not (DWI_PATH and BVAL_PATH and BVECS_PATH):
    raise SystemExit("❌ DWI.nii(.gz) / bvals / bvecs が見つかりません。Driveの場所を教えるか、DICOMをzipでアップロードしてください。")

print("DWI :", DWI_PATH)
print("bvals:", BVAL_PATH)
print("bvecs:", BVECS_PATH)

# 6) DIPYで u=0 Baseline 生成（制御なし・同じ種点・推定ステップ長）
data, affine = load_nifti(DWI_PATH)
bvals = np.loadtxt(BVAL_PATH); bvecs = np.loadtxt(BVECS_PATH)
gtab = gradient_table(bvals, bvecs)
masked, brainmask = median_otsu(data, numpass=2)
response, ratio   = auto_response_ssst(gtab, masked, roi_radius=10, fa_thr=0.7)
csd = ConstrainedSphericalDeconvModel(gtab, response, sh_order=8)
peaks = peaks_from_model(csd, masked, default_sphere,
                         relative_peak_threshold=0.8, min_separation_angle=25,
                         mask=brainmask, npeaks=5, normalize_peaks=True)
stop = BinaryStoppingCriterion(peaks.gfa > 0.2)

streamlines_u0 = Streamlines(LocalTracking(
    peaks, stop, seeds_world, affine, step_size=float(STEP_MM), max_cross=1, return_all=False
))

os.makedirs("/content/baseline_eval", exist_ok=True)
BASE_TRK = "/content/baseline_eval/baseline_u0_sameSeeds.trk"
save_trk(streamlines_u0, BASE_TRK, affine=affine, shape=data.shape[:3])
print(f"[saved baseline] {BASE_TRK}  n={len(streamlines_u0)}")

# 7) 指標（長さ/平均曲率）を読み出し、CGTAD と重ねヒスト → PDF と LaTeXマクロ
def load_len_curv(trk):
    obj = sl_load(trk); Ls, Ks = [], []
    for s in obj.streamlines:
        s = np.asarray(s, dtype=np.float64)
        if s.shape[0] >= 2:
            d = np.diff(s, axis=0); Ls.append(float(np.sum(np.linalg.norm(d, axis=1))))
        if s.shape[0] >= 3:
            p0,p1,p2=s[:-2],s[1:-1],s[2:]; v1=p1-p0; v2=p2-p1
            L1=np.linalg.norm(v1,axis=1)+1e-9; L2=np.linalg.norm(v2,axis=1)+1e-9
            ct=np.clip(np.sum(v1*v2,axis=1)/(L1*L2),-1,1); th=np.arccos(ct); smean=0.5*(L1+L2)+1e-9
            k=2.0*np.sin(0.5*th)/smean
            if k.size: Ks.append(float(np.mean(k)))
    return np.asarray(Ls), np.asarray(Ks)

Lb,Kb = load_len_curv(BASE_TRK)
Lc,Kc = load_len_curv(CGTAD_TRK)

def plot_overlay(a,b,title,xlabel,out,bins=64):
    lo = min(float(np.min(a)) if a.size else 0, float(np.min(b)) if b.size else 0)
    hi = max(float(np.max(a)) if a.size else 1, float(np.max(b)) if b.size else 1)
    if not np.isfinite(lo) or not np.isfinite(hi) or lo==hi: lo,hi=0.0,max(1.0,hi if np.isfinite(hi) else 1.0)
    edges = np.linspace(lo, hi, bins+1)
    plt.figure(figsize=(6,5))
    plt.hist(a, bins=edges, alpha=0.5, label="Baseline (u=0)")
    plt.hist(b, bins=edges, alpha=0.5, label="CGTAD")
    plt.title(title); plt.xlabel(xlabel); plt.ylabel("count"); plt.legend()
    os.makedirs(os.path.dirname(out), exist_ok=True)
    plt.tight_layout(); plt.savefig(out, bbox_inches="tight"); plt.close()

os.makedirs("/content/compare_eval", exist_ok=True)
plot_overlay(Lb,Lc,"Length distribution (u=0 vs CGTAD)","Length (mm)","/content/compare_eval/overlay_Length_mm.pdf")
plot_overlay(Kb,Kc,"Curvature distribution (u=0 vs CGTAD)","Curvature (1/mm)","/content/compare_eval/overlay_Curvature_1_mm.pdf")
print("[saved] /content/compare_eval/overlay_Length_mm.pdf")
print("[saved] /content/compare_eval/overlay_Curvature_1_mm.pdf")

def fmt(v,p=3):
    if not np.isfinite(v): return '---'
    if v!=0 and (abs(v)<1e-4 or abs(v)>=1e4): return f"{v:.{p}e}"
    return f"{v:.{p}f}"
def dpp(a,b):
    if not (np.isfinite(a) and np.isfinite(b)) or a==0: return '---'
    return f"{100.0*(b-a)/abs(a):.1f}"
def summarize(arr):
    return dict(mean=float(np.mean(arr)) if arr.size else float("nan"),
                std=float(np.std(arr)) if arr.size else float("nan"),
                n=int(arr.size))

summs = {"length_mm":{"base":summarize(Lb),"cgtad":summarize(Lc)},
         "curv_1_per_mm":{"base":summarize(Kb),"cgtad":summarize(Kc)}}

os.makedirs("tex", exist_ok=True)
lines = [
  "% Auto-generated: Baseline(u=0 same seeds) vs CGTAD",
  r"\providecommand{\CompLenBase}{%s}"     % fmt(summs["length_mm"]["base"]["mean"],2),
  r"\providecommand{\CompLenCtl}{%s}"      % fmt(summs["length_mm"]["cgtad"]["mean"],2),
  r"\providecommand{\CompLenDeltaPct}{%s}" % dpp(summs["length_mm"]["base"]["mean"], summs["length_mm"]["cgtad"]["mean"]),
  r"\providecommand{\CompCurvBase}{%s}"    % fmt(summs["curv_1_per_mm"]["base"]["mean"],3),
  r"\providecommand{\CompCurvCtl}{%s}"     % fmt(summs["curv_1_per_mm"]["cgtad"]["mean"],3),
  r"\providecommand{\CompCurvDeltaPct}{%s}"% dpp(summs["curv_1_per_mm"]["base"]["mean"], summs["curv_1_per_mm"]["cgtad"]["mean"]),
]
open("tex/compare_macros_generated.tex","w",encoding="utf-8").write("\n".join(lines)+"\n")
print("[macros] tex/compare_macros_generated.tex")


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
[CGTAD] /content/B_streamlines.trk
[seeds] 197959  [estimated step] 0.500 mm
Mounted at /content/drive


SystemExit: ❌ DWI.nii(.gz) / bvals / bvecs が見つかりません。Driveの場所を教えるか、DICOMをzipでアップロードしてください。

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
from google.colab import drive; import os, glob
if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

roots = ['/content','/content/drive/MyDrive','/content/drive/Shareddrives']
def scan(patterns):
    out=[]
    for r in roots:
        for p in patterns:
            out += glob.glob(os.path.join(r,'**',p), recursive=True)
    return sorted(set(out))

nii  = scan(['*dwi*.nii*','*data.nii*'])
bval = scan(['*.bval','*bval*'])
bvec = scan(['*.bvec','*bvec*'])
print('NIfTI:', nii[:10]); print('bvals:', bval[:10]); print('bvecs:', bvec[:10])


NIfTI: []
bvals: []
bvecs: []


In [6]:
# === 強化サーチ：NIfTI と bvals/bvecs を広めに探す ===
from google.colab import drive; import os, glob
if not os.path.ismount('/content/drive'): drive.mount('/content/drive')

roots = ['/content','/content/drive/MyDrive','/content/drive/Shareddrives']
nii_pats  = ['*dwi*.nii*','*diff*.nii*','*DTI*.nii*','*data*.nii*','*nii.gz']
bval_pats = ['*.bval','*bval*','*bvals*']
bvec_pats = ['*.bvec','*bvec*','*bvecs*']

def find(patterns):
    hits=set()
    for r in roots:
        for p in patterns:
            hits.update(glob.glob(os.path.join(r,'**',p), recursive=True))
    return sorted(hits)

def show(hits, tag):
    from pathlib import Path
    print(f"\n[{tag}] {len(hits)} files")
    for p in hits[:20]:  # 20件まで
        try:
            sz = os.path.getsize(p)/1e6
            print(f"{sz:6.1f} MB  {p}")
        except: print(p)

nii  = find(nii_pats);  show(nii,'NIfTI candidates')
bval = find(bval_pats); show(bval,'bvals candidates')
bvec = find(bvec_pats); show(bvec,'bvecs candidates')



[NIfTI candidates] 0 files

[bvals candidates] 0 files

[bvecs candidates] 0 files


In [7]:
from nibabel.streamlines import load
import numpy as np, os, glob, json

p = "/content/drive/MyDrive/CGTAD/B_streamlines.trk"  # 置いた場所に合わせて
obj = load(p)
hdr = obj.header

print("voxsize:", tuple(float(x) for x in hdr["voxel_sizes"]))
print("dim    :", tuple(int(x) for x in hdr["dim"]))
print("n_streamlines:", len(obj.streamlines))

# 代表的な推定：1.25mmならHCP、~2.0mmなら(Stanford) HARDI 系の可能性大
vs = float(hdr["voxel_sizes"][0])
if 1.20 <= vs <= 1.30:
    print("→ たぶん HCP 解像度")
elif 1.9 <= vs <= 2.1:
    print("→ たぶん HARDI(2mm) 系")
else:
    print("→ 別解像度（プロトコルを確認）")

# だれが生成したかの手掛かり（保存元セル探索）
!grep -R --line-number -e "B_streamlines.trk" -e "save_trk" /content || true
!find /content -type f -name "B_streamlines.trk" -printf "%TY-%Tm-%Td %TH:%TM  %p\n" | sort


voxsize: (2.0, 2.0, 2.0)


KeyError: 'dim'

In [8]:
from nibabel.streamlines import load
p = "/content/drive/MyDrive/CGTAD/B_streamlines.trk"  # パスは合わせて

obj = load(p)
hdr = obj.header

# まずキー一覧を確認
print("keys:", sorted(hdr.keys()))

# 安全に取得
voxsize = tuple(float(x) for x in hdr.get('voxel_sizes', hdr.get('voxel_size', [])))
dims    = tuple(int(x)   for x in hdr.get('dimensions',  hdr.get('dim', [])))
n_strm  = len(obj.streamlines)

print("voxel_sizes:", voxsize)
print("dimensions :", dims)
print("n_streamlines:", n_strm)

# ざっくり判定
if voxsize and 1.20 <= voxsize[0] <= 1.30:
    print("→ HCP 解像度の可能性が高い")
elif voxsize and 1.9 <= voxsize[0] <= 2.1:
    print("→ HARDI(2mm) 系の可能性が高い")
else:
    print("→ いずれでもない解像度（元データを要確認）")


keys: ['_offset_data', 'dimensions', 'endianness', 'hdr_size', 'image_orientation_patient', 'invert_x', 'invert_y', 'invert_z', 'magic_number', 'nb_properties_per_streamline', 'nb_scalars_per_point', 'nb_streamlines', 'origin', 'pad1', 'pad2', 'property_name', 'reserved', 'scalar_name', 'swap_xy', 'swap_yz', 'swap_zx', 'version', 'voxel_order', 'voxel_sizes', 'voxel_to_rasmm']
voxel_sizes: (2.0, 2.0, 2.0)
dimensions : (81, 106, 76)
n_streamlines: 204419
→ HARDI(2mm) 系の可能性が高い


In [11]:
import numpy as np
from nibabel.streamlines import load as trk_load

def _len_mm(s):
    if s.shape[0] < 2: return 0.0
    d = np.diff(s, axis=0)
    return float(np.linalg.norm(d, axis=1).sum())

def _curv_mean(s, eps=1e-9):
    if s.shape[0] < 3: return np.nan
    p0,p1,p2 = s[:-2], s[1:-1], s[2:]
    v1 = p1 - p0; v2 = p2 - p1
    L1 = np.linalg.norm(v1, axis=1) + eps
    L2 = np.linalg.norm(v2, axis=1) + eps
    ct = np.clip(np.sum(v1*v2, axis=1) / (L1*L2), -1, 1)
    th = np.arccos(ct); smean = 0.5*(L1+L2) + eps
    k = 2.0*np.sin(0.5*th)/smean  # 1/mm
    return float(np.nanmean(k)) if k.size else np.nan

def extract_arrays(trk_path):
    """trk から (長さmm配列, 曲率1/mm配列) を返す"""
    obj = trk_load(trk_path)
    Ls, Ks = [], []
    for s in obj.streamlines:
        s = np.asarray(s, dtype=np.float64)
        if s.shape[0] >= 2:
            Ls.append(_len_mm(s))
        if s.shape[0] >= 3:
            kv = _curv_mean(s)
            if np.isfinite(kv): Ks.append(kv)
    return np.array(Ls, dtype=float), np.array(Ks, dtype=float)


In [13]:
# こういう誤記を探します
import re, inspect
print(re.findall(r"np\.array\(\s*\[,\s*float", open('/content/*.py','r').read() ))


FileNotFoundError: [Errno 2] No such file or directory: '/content/*.py'

In [14]:
# === Stanford HARDI を用いた u=0 ベースライン生成（同じ seeds）+ 比較図・TeX 一発セル ===
# 依存
import sys, os, json, numpy as np, matplotlib.pyplot as plt
from nibabel.streamlines import load as trk_load
import nibabel as nib

# DIPY が使えない場合だけ最小限インストール（numpyは現状維持）
try:
    import dipy
except Exception:
    !pip -q install dipy nibabel scikit-image scipy

from dipy.data import fetch_stanford_hardi, read_stanford_hardi
from dipy.segment.mask import median_otsu
from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel, auto_response_ssst
from dipy.direction import peaks_from_model
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.streamline import Streamlines
from dipy.tracking.stopping_criterion import BinaryStoppingCriterion
from dipy.io.streamline import save_trk

# ---------- ユーザ環境の CGTAD .trk ----------
CGTAD_TRK = "/content/drive/MyDrive/CGTAD/B_streamlines.trk"  # 必要なら変更

# ---------- .trk 読み込み＆step推定＆seed抽出 ----------
obj = trk_load(CGTAD_TRK)
hdr = obj.header
streams = [np.asarray(s, dtype=np.float64) for s in obj.streamlines]
n_str = len(streams)
voxsize = tuple(float(x) for x in hdr.get('voxel_sizes', hdr.get('voxel_size', [])))
print(f"[CGTAD] {CGTAD_TRK}")
print(f"        n={n_str}, vox={voxsize}")

# 推定 step（平均セグメント長）
def est_step(ss):
    lens=[]
    for s in ss:
        if s.shape[0] < 2: continue
        d = np.diff(s, axis=0)
        lens += list(np.linalg.norm(d, axis=1))
    if not lens: return 0.5
    m = float(np.median(lens))
    return max(0.25, min(1.5, round(m, 3)))

STEP_MM = est_step(streams)
print(f"[seeds] estimated step ~ {STEP_MM} mm")

# seeds（各ストリームの開始点, world(mm)）
seeds_world = np.array([s[0] for s in streams if len(s)>=1], dtype=np.float64)
print(f"[seeds] count = {seeds_world.shape[0]}")

# ---------- Stanford HARDI を取得 ----------
fetch_stanford_hardi()
data, affine, gtab = read_stanford_hardi()
print("[HARDI] data shape:", data.shape, "vox:", tuple(np.sqrt((affine[:3,:3]**2).sum(0))))

# seeds の妥当性チェック（画像内に入っているか）
ijk = nib.affines.apply_affine(np.linalg.inv(affine), seeds_world)
inside = (
    (ijk[:,0] >= 0) & (ijk[:,0] < data.shape[0]) &
    (ijk[:,1] >= 0) & (ijk[:,1] < data.shape[1]) &
    (ijk[:,2] >= 0) & (ijk[:,2] < data.shape[2])
)
valid_seeds = seeds_world[inside]
drop = seeds_world.shape[0] - valid_seeds.shape[0]
print(f"[seeds] valid {valid_seeds.shape[0]} / dropped {drop}")

# 妥当な seeds が少なすぎる場合は mask ベースにフォールバック
if valid_seeds.shape[0] < 1000:
    print("[warn] seeds が画像外に多いのでフォールバック（脳マスクから均一seeding）")
    data_masked, mask = median_otsu(data, vol_idx=range(data.shape[3]), numpass=2)
    # 粗密は必要に応じ調整
    se = np.argwhere(mask)[::4]  # 4ボクセルに1点
    # voxel→world
    valid_seeds = nib.affines.apply_affine(affine, se)

# ---------- CSD モデル & 方向場 ----------
data_masked, mask = median_otsu(data, vol_idx=range(data.shape[3]), numpass=2)
response, ratio = auto_response_ssst(gtab, data_masked, roi_radius=10, fa_thr=0.7)
csd_model = ConstrainedSphericalDeconvModel(gtab, response)
peaks = peaks_from_model(csd_model, data_masked, sphere=None,
                         relative_peak_threshold=0.5,
                         min_separation_angle=25, mask=mask,
                         return_sh=False, normalize_peaks=True)
stop = BinaryStoppingCriterion(mask)

# ---------- LocalTracking（u=0 ベースライン） ----------
stream_gen = LocalTracking(peaks, stop, valid_seeds, affine,
                           step_size=float(STEP_MM), max_cross=1,
                           return_all=False)
baseline = Streamlines(stream_gen)

# 保存
os.makedirs("/content/baseline_eval", exist_ok=True)
BASE_TRK = "/content/baseline_eval/baseline_u0_sameSeeds.trk"
save_trk(BASE_TRK, baseline, affine=affine)
print(f"[saved baseline] {BASE_TRK}  (n={len(baseline)})")

# ---------- 評価：長さ & 曲率 ----------
def len_mm(s):
    if s.shape[0] < 2: return 0.0
    d = np.diff(s,axis=0)
    return float(np.sum(np.linalg.norm(d,axis=1)))
def curv_mean(s, eps=1e-9):
    if s.shape[0] < 3: return np.nan
    p0,p1,p2 = s[:-2], s[1:-1], s[2:]
    v1 = p1-p0; v2 = p2-p1
    L1 = np.linalg.norm(v1,axis=1)+eps; L2 = np.linalg.norm(v2,axis=1)+eps
    ct = np.clip(np.sum(v1*v2,axis=1)/(L1*L2), -1, 1)
    th = np.arccos(ct); smean = 0.5*(L1+L2)+eps
    k = 2.0*np.sin(0.5*th)/smean  # 1/mm
    return float(np.nanmean(k)) if k.size else np.nan

def extract_arrays(trk_path):
    obj = trk_load(trk_path)
    L=[]; K=[]
    for s in obj.streamlines:
        s=np.asarray(s, dtype=np.float64)
        if s.shape[0] < 2: continue
        L.append(len_mm(s))
        if s.shape[0] >= 3:
            kv = curv_mean(s)
            if np.isfinite(kv): K.append(kv)
    return np.array(L, np.array(K, float)

Lb, Kb = extract_arrays(BASE_TRK)
Lc, Kc = extract_arrays(CGTAD_TRK)

print("[auto] baseline:", len(Lb), "   CGTAD:", len(Lc))

# ---------- 図（重ねヒスト） ----------
def plot_overlay(a,b, title, xl, out, bins=64):
    plt.figure(figsize=(6.4,5))
    lo = float(np.nanmin(np.r_[a,b])) if a.size and b.size else 0.0
    hi = float(np.nanmax(np.r_[a,b])) if a.size and b.size else 1.0
    lo, hi = max(lo,0.0), max(hi,1e-6)
    edges = np.linspace(lo, hi, bins+1)
    plt.hist(a, bins=edges, alpha=0.6, label="Baseline (u=0)")
    plt.hist(b, bins=edges, alpha=0.6, label="CGTAD")
    plt.title(title); float), plt.xlabel(xl); plt.ylabel("count"); plt.legend()
    os.makedirs(os.path.dirname(out), exist_ok=True)
    plt.tight_layout(); plt.savefig(out, bbox_inches="tight"); plt.close()

plot_overlay(Lb,Lc, "Length distribution (Baseline vs CGTAD)", "Length (mm)",
             "/content/compare_eval/overlay_Length_mm.pdf")
plot_overlay(Kb,Kc, "Curvature distribution (Baseline vs CGTAD)", "Curvature (1/mm)",
             "/content/compare_eval/overlay_curvature_1_mm.pdf")
print("[saved] /content/compare_eval/overlay_Length_mm.pdf")
print("[saved] /content/compare_eval/overlay_curvature_1_mm.pdf")

# ---------- LaTeX マクロ ----------
def summarize(arr):
    arr = arr[np.isfinite(arr)]
    return dict(mean=float(np.mean(arr)) if arr.size else float("nan"),
                std =float(np.std(arr))  if arr.size else float("nan"),
                n=int(arr.size))
sums = {
  "length_mm": {"base": summarize(Lb), "cgtad": summarize(Lc)},
  "curv_1_per_mm": {"base": summarize(Kb), "cgtad": summarize(Kc)}
}
os.makedirs("tex", exist_ok=True)
lines = []
lines.append("% Auto-generated: Baseline(u=0) vs CGTAD")
def fmt(x,p=3):
    if not np.isfinite(x): return '---'
    return f"{x:.{p}f}" if (1e-4 <= abs(x) < 1e4) else f"{x:.{p}e}"
lines += [
  rf"\providecommand{{\CompNStreams}}{{{len(Lc)}}}",
  rf"\providecommand{{\CompLenMeanBase}}{{{fmt(sums['length_mm']['base']['mean'],2)}}}",
  rf"\providecommand{{\CompLenMeanCG}}{{{fmt(sums['length_mm']['cgtad']['mean'],2)}}}",
  rf"\providecommand{{\CompCurvMeanBase}}{{{fmt(sums['curv_1_per_mm']['base']['mean'],3)}}}",
  rf"\providecommand{{\CompCurvMeanCG}}{{{fmt(sums['curv_1_per_mm']['cgtad']['mean'],3)}}}",
]
open("tex/compare_macros_generated.tex","w",encoding="utf-8").write("\n".join(lines)+"\n")
print("[macros] tex/compare_macros_generated.tex")


SyntaxError: invalid syntax. Perhaps you forgot a comma? (ipython-input-2966537099.py, line 124)

In [ ]:
# === AM reset: DIPY/CSD 用の安定セットに固定 → 強制再起動（Colab） ===
# ポイント:
# - NumPyは1.26系に固定（2.xは使わない）
# - SciPyは1.10.1、pandasは2.2.2に固定
# - OpenCVはheadlessの安定版に固定（GUI不要ならこれで十分）

%pip -q install --no-input --upgrade --force-reinstall \
  numpy==1.26.4 \
  scipy==1.10.1 \
  pandas==2.2.2 \
  nibabel==5.1.0 \
  dipy==1.8.0 \
  statsmodels==0.14.2 \
  matplotlib==3.8.4 \
  scikit-image==0.21.0 \
  opencv-python-headless==4.9.0.80

import os, signal
print("Pinned versions installed. Restarting runtime to load compiled extensions...")
os.kill(os.getpid(), signal.SIGKILL)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
arviz 0.22.0 requires scipy>=1.11.0, but you have scipy 1.10.1 which is incompatible.
jax 0.5.3 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
xarray-einstats 0.9.1 requires scipy>=1.11, but you have scipy 1.10.1 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jaxlib 0.5.3 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
cvxpy 1.6.7 requires scipy>=1.11.0, but you have scipy 1.10.1 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you hav

In [1]:
# === リセット一発セル：u=0 Baseline 生成 → CGTADと比較（Colab） ===
# 1) 依存とツール
!pip -q install -U numpy scipy nibabel dipy statsmodels pandas scikit-image
!apt-get -qq update && apt-get -qq install -y dcm2niix >/dev/null

import os, re, glob, json, numpy as np, matplotlib.pyplot as plt
from nibabel.streamlines import load as sl_load
from dipy.io.image import load_nifti
from dipy.core.gradients import gradient_table
from dipy.segment.mask import median_otsu
from dipy.reconst.csdeconv import auto_response_ssst, ConstrainedSphericalDeconvModel
from dipy.direction import peaks_from_model
from dipy.data import default_sphere
from dipy.tracking.stopping_criterion import BinaryStoppingCriterion
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.streamline import Streamlines
from dipy.io.streamline import save_trk

# 2) 設定（ここだけ編集）：あなたの CGTAD .trk の実パス
CGTAD_TRK = "/content/CGTAD_runs_20250818_0634/B_streamlines.trk"  # ←必要に応じて修正

# 3) CGTAD が見つからなければ自動探索
if not os.path.exists(CGTAD_TRK):
    cands = glob.glob("/content/**/*.trk", recursive=True)
    cands = [p for p in cands if re.search(r"(B_stream|cgtad|opt)", os.path.basename(p), re.I)]
    if not cands:
        raise SystemExit("❌ CGTAD .trk が見つかりません。CGTAD_TRK を実パスに直してください。")
    CGTAD_TRK = sorted(cands)[0]
print("[CGTAD]", CGTAD_TRK)

# 4) CGTAD の種点（最初の頂点）を seeds に採用、ステップ長も推定
obj = sl_load(CGTAD_TRK)
streams = [np.asarray(s, dtype=np.float64) for s in obj.streamlines if len(s)>=2]
seeds_world = [s[0] for s in streams]
# ステップ長の中央値（mm）
def step_median_mm(s):
    d = np.diff(s, axis=0);
    return float(np.median(np.linalg.norm(d, axis=1))) if d.size else np.nan
STEP_MM = float(np.nanmedian([step_median_mm(s) for s in streams]))
if not np.isfinite(STEP_MM): STEP_MM = 0.5
print(f"[seeds] {len(seeds_world)}  [estimated step] {STEP_MM:.3f} mm")

# 5) DWI の取得：既存NIfTIを探す → 無ければ Drive から DICOM候補→自動変換
DWI_PATH=BVAL_PATH=BVECS_PATH=None
# 5-1) 既存のNIfTI三点セットを /content と Drive から検索
roots = ["/content", "/content/drive/MyDrive", "/content/drive/Shareddrives"]
def find_files(patterns):
    out=[]
    for r in roots:
        for pat in patterns:
            out += glob.glob(os.path.join(r, "**", pat), recursive=True)
    return sorted(set(out))
nii  = find_files(["*dwi*.nii*", "*data.nii*"])
bval = find_files(["*.bval", "*bval*"])
bvec = find_files(["*.bvec", "*bvec*"])
if nii and bval and bvec:
    DWI_PATH, BVAL_PATH, BVECS_PATH = nii[0], bval[0], bvec[0]

# 5-2) 無ければ Drive をマウントして DICOM を変換
if not (DWI_PATH and BVAL_PATH and BVECS_PATH):
    try:
        from google.colab import drive
        if not os.path.ismount("/content/drive"):
            drive.mount("/content/drive")
    except Exception:
        pass
    # DICOM候補（枚数が多いフォルダ）
    cands=[]
    for root in roots[1:]:
        for dp, dn, fn in os.walk(root):
            n=0
            for f in fn:
                fl=f.lower()
                if fl.endswith((".dcm",".ima",".dicom")) or bool(re.fullmatch(r"\d{5,}", f)):
                    n+=1
            if n>=30: cands.append((n,dp))
    cands.sort(reverse=True)
    if cands:
        DICOM_DIR=cands[0][1]
        print("[DICOM candidate]", DICOM_DIR, f"(files≈{cands[0][0]})")
        os.makedirs("/content/dwi_out", exist_ok=True)
        # 変換
        get_ipython().system('dcm2niix -ba y -z y -o /content/dwi_out -f dwi "{DICOM_DIR}"')
        nii  = sorted(glob.glob("/content/dwi_out/*.nii*"))
        bval = sorted(glob.glob("/content/dwi_out/*.bval"))
        bvec = sorted(glob.glob("/content/dwi_out/*.bvec"))
        if nii and bval and bvec:
            DWI_PATH, BVAL_PATH, BVECS_PATH = nii[0], bval[0], bvec[0]

if not (DWI_PATH and BVAL_PATH and BVECS_PATH):
    raise SystemExit("❌ DWI.nii(.gz) / bvals / bvecs が見つかりません。Driveの場所を教えるか、DICOMをzipでアップロードしてください。")

print("DWI :", DWI_PATH)
print("bvals:", BVAL_PATH)
print("bvecs:", BVECS_PATH)

# 6) DIPYで u=0 Baseline 生成（制御なし・同じ種点・推定ステップ長）
data, affine = load_nifti(DWI_PATH)
bvals = np.loadtxt(BVAL_PATH); bvecs = np.loadtxt(BVECS_PATH)
gtab = gradient_table(bvals, bvecs)
masked, brainmask = median_otsu(data, numpass=2)
response, ratio   = auto_response_ssst(gtab, masked, roi_radius=10, fa_thr=0.7)
csd = ConstrainedSphericalDeconvModel(gtab, response, sh_order=8)
peaks = peaks_from_model(csd, masked, default_sphere,
                         relative_peak_threshold=0.8, min_separation_angle=25,
                         mask=brainmask, npeaks=5, normalize_peaks=True)
stop = BinaryStoppingCriterion(peaks.gfa > 0.2)

streamlines_u0 = Streamlines(LocalTracking(
    peaks, stop, seeds_world, affine, step_size=float(STEP_MM), max_cross=1, return_all=False
))

os.makedirs("/content/baseline_eval", exist_ok=True)
BASE_TRK = "/content/baseline_eval/baseline_u0_sameSeeds.trk"
save_trk(streamlines_u0, BASE_TRK, affine=affine, shape=data.shape[:3])
print(f"[saved baseline] {BASE_TRK}  n={len(streamlines_u0)}")

# 7) 指標（長さ/平均曲率）を読み出し、CGTAD と重ねヒスト → PDF と LaTeXマクロ
def load_len_curv(trk):
    obj = sl_load(trk); Ls, Ks = [], []
    for s in obj.streamlines:
        s = np.asarray(s, dtype=np.float64)
        if s.shape[0] >= 2:
            d = np.diff(s, axis=0); Ls.append(float(np.sum(np.linalg.norm(d, axis=1))))
        if s.shape[0] >= 3:
            p0,p1,p2=s[:-2],s[1:-1],s[2:]; v1=p1-p0; v2=p2-p1
            L1=np.linalg.norm(v1,axis=1)+1e-9; L2=np.linalg.norm(v2,axis=1)+1e-9
            ct=np.clip(np.sum(v1*v2,axis=1)/(L1*L2),-1,1); th=np.arccos(ct); smean=0.5*(L1+L2)+1e-9
            k=2.0*np.sin(0.5*th)/smean
            if k.size: Ks.append(float(np.mean(k)))
    return np.asarray(Ls), np.asarray(Ks)

Lb,Kb = load_len_curv(BASE_TRK)
Lc,Kc = load_len_curv(CGTAD_TRK)

def plot_overlay(a,b,title,xlabel,out,bins=64):
    lo = min(float(np.min(a)) if a.size else 0, float(np.min(b)) if b.size else 0)
    hi = max(float(np.max(a)) if a.size else 1, float(np.max(b)) if b.size else 1)
    if not np.isfinite(lo) or not np.isfinite(hi) or lo==hi: lo,hi=0.0,max(1.0,hi if np.isfinite(hi) else 1.0)
    edges = np.linspace(lo, hi, bins+1)
    plt.figure(figsize=(6,5))
    plt.hist(a, bins=edges, alpha=0.5, label="Baseline (u=0)")
    plt.hist(b, bins=edges, alpha=0.5, label="CGTAD")
    plt.title(title); plt.xlabel(xlabel); plt.ylabel("count"); plt.legend()
    os.makedirs(os.path.dirname(out), exist_ok=True)
    plt.tight_layout(); plt.savefig(out, bbox_inches="tight"); plt.close()

os.makedirs("/content/compare_eval", exist_ok=True)
plot_overlay(Lb,Lc,"Length distribution (u=0 vs CGTAD)","Length (mm)","/content/compare_eval/overlay_Length_mm.pdf")
plot_overlay(Kb,Kc,"Curvature distribution (u=0 vs CGTAD)","Curvature (1/mm)","/content/compare_eval/overlay_Curvature_1_mm.pdf")
print("[saved] /content/compare_eval/overlay_Length_mm.pdf")
print("[saved] /content/compare_eval/overlay_Curvature_1_mm.pdf")

def fmt(v,p=3):
    if not np.isfinite(v): return '---'
    if v!=0 and (abs(v)<1e-4 or abs(v)>=1e4): return f"{v:.{p}e}"
    return f"{v:.{p}f}"
def dpp(a,b):
    if not (np.isfinite(a) and np.isfinite(b)) or a==0: return '---'
    return f"{100.0*(b-a)/abs(a):.1f}"
def summarize(arr):
    return dict(mean=float(np.mean(arr)) if arr.size else float("nan"),
                std=float(np.std(arr)) if arr.size else float("nan"),
                n=int(arr.size))

summs = {"length_mm":{"base":summarize(Lb),"cgtad":summarize(Lc)},
         "curv_1_per_mm":{"base":summarize(Kb),"cgtad":summarize(Kc)}}

os.makedirs("tex", exist_ok=True)
lines = [
  "% Auto-generated: Baseline(u=0 same seeds) vs CGTAD",
  r"\providecommand{\CompLenBase}{%s}"     % fmt(summs["length_mm"]["base"]["mean"],2),
  r"\providecommand{\CompLenCtl}{%s}"      % fmt(summs["length_mm"]["cgtad"]["mean"],2),
  r"\providecommand{\CompLenDeltaPct}{%s}" % dpp(summs["length_mm"]["base"]["mean"], summs["length_mm"]["cgtad"]["mean"]),
  r"\providecommand{\CompCurvBase}{%s}"    % fmt(summs["curv_1_per_mm"]["base"]["mean"],3),
  r"\providecommand{\CompCurvCtl}{%s}"     % fmt(summs["curv_1_per_mm"]["cgtad"]["mean"],3),
  r"\providecommand{\CompCurvDeltaPct}{%s}"% dpp(summs["curv_1_per_mm"]["base"]["mean"], summs["curv_1_per_mm"]["cgtad"]["mean"]),
]
open("tex/compare_macros_generated.tex","w",encoding="utf-8").write("\n".join(lines)+"\n")
print("[macros] tex/compare_macros_generated.tex")


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.2 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.2 which is incompatib

ImportError: cannot import name '_center' from 'numpy.core._multiarray_umath' (/usr/local/lib/python3.11/dist-packages/numpy/core/_multiarray_umath.cpython-311-x86_64-linux-gnu.so)

In [2]:
# 最低限に固定（Colabと互換が取れる組み合わせ）
!pip -q install --force-reinstall --no-cache-dir \
  "numpy==1.26.4" "pandas==2.2.2" "scipy==1.11.4" \
  "nibabel==5.2.1" "dipy==1.7.0" "scikit-image==0.22.0"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 286.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 318.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 315.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 291.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 151.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 249.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 356.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 427.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 423.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66

In [3]:
import numpy as np, dipy, nibabel as nib, scipy
print("numpy", np.__version__, "| dipy", dipy.__version__, "| nibabel", nib.__version__, "| scipy", scipy.__version__)


numpy 1.26.4 | dipy 1.11.0 | nibabel 5.3.2 | scipy 1.16.1


In [5]:
import numpy as np
from nibabel.streamlines import load as trk_load

def _len_mm(s):
    if s.shape[0] < 2: return 0.0
    d = np.diff(s, axis=0)
    return float(np.linalg.norm(d, axis=1).sum())

def _curv_mean(s, eps=1e-9):
    if s.shape[0] < 3: return np.nan
    p0,p1,p2 = s[:-2], s[1:-1], s[2:]
    v1 = p1 - p0; v2 = p2 - p1
    L1 = np.linalg.norm(v1, axis=1) + eps
    L2 = np.linalg.norm(v2, axis=1) + eps
    ct = np.clip(np.sum(v1*v2, axis=1) / (L1*L2), -1, 1)
    th = np.arccos(ct); smean = 0.5*(L1+L2) + eps
    k = 2.0*np.sin(0.5*th)/smean  # 1/mm
    return float(np.nanmean(k)) if k.size else np.nan

def extract_arrays(trk_path):
    """trk から (長さmm配列, 曲率1/mm配列) を返す"""
    obj = trk_load(trk_path)
    Ls, Ks = [], []
    for s in obj.streamlines:
        s = np.asarray(s, dtype=np.float64)
        if s.shape[0] >= 2:
            Ls.append(_len_mm(s))
        if s.shape[0] >= 3:
            kv = _curv_mean(s)
            if np.isfinite(kv): Ks.append(kv)
    return np.array(Ls, dtype=float), np.array(Ks, dtype=float)


In [6]:
# こういう誤記を探します
import re, inspect
print(re.findall(r"np\.array\(\s*\[,\s*float", open('/content/*.py','r').read() ))


FileNotFoundError: [Errno 2] No such file or directory: '/content/*.py'

In [7]:
# ← これはダメ：open('/content/*.py')
# 置き換え（必要なときだけ実行）:
import glob, re
hits=[]
for p in glob.glob('/content/**/*.py', recursive=True):
    try:
        txt = open(p, 'r', encoding='utf-8', errors='ignore').read()
        if re.search(r"np\.array\(\s*\[,\s*float", txt):
            hits.append(p)
    except Exception:
        pass
print(hits or "no hits")


no hits


In [8]:
import os, numpy as np, matplotlib.pyplot as plt
from nibabel.streamlines import load as trk_load

BASE_TRK  = "/content/baseline_eval/baseline_u0_sameSeeds.trk"  # 生成済みのu=0
CGTAD_TRK = "/content/drive/MyDrive/CGTAD/B_streamlines.trk"    # あなたのCGTAD

assert os.path.exists(CGTAD_TRK), "CGTAD .trk が見つかりません"
assert os.path.exists(BASE_TRK),  "ベースライン .trk が見つかりません（先に生成を）"

# 上で定義した extract_arrays を利用
Lb, Kb = extract_arrays(BASE_TRK)
Lc, Kc = extract_arrays(CGTAD_TRK)
print("[auto] baseline:", len(Lb), " CGTAD:", len(Lc))

def plot_overlay(a,b,title,xlabel,out,bins=64):
    plt.figure(figsize=(6.4,5))
    lo = float(np.nanmin(np.r_[a,b])) if a.size and b.size else 0.0
    hi = float(np.nanmax(np.r_[a,b])) if a.size and b.size else 1.0
    lo, hi = max(lo,0.0), max(hi,1e-6)
    edges = np.linspace(lo, hi, bins+1)
    plt.hist(a, bins=edges, alpha=0.6, label="Baseline (u=0)")
    plt.hist(b, bins=edges, alpha=0.6, label="CGTAD")
    plt.title(title); plt.xlabel(xlabel); plt.ylabel("count"); plt.legend()
    os.makedirs(os.path.dirname(out), exist_ok=True)
    plt.tight_layout(); plt.savefig(out, bbox_inches="tight"); plt.close()

plot_overlay(Lb,Lc, "Length distribution (Baseline vs CGTAD)", "Length (mm)",
             "/content/compare_eval/overlay_Length_mm.pdf")
plot_overlay(Kb,Kc, "Curvature distribution (Baseline vs CGTAD)", "Curvature (1/mm)",
             "/content/compare_eval/overlay_curvature_1_mm.pdf")
print("[saved] /content/compare_eval/overlay_Length_mm.pdf")
print("[saved] /content/compare_eval/overlay_curvature_1_mm.pdf")

# LaTeX マクロ
def summarize(arr):
    arr = arr[np.isfinite(arr)]
    return dict(mean=float(np.mean(arr)) if arr.size else float("nan"),
                std =float(np.std(arr))  if arr.size else float("nan"),
                n=int(arr.size))
sums = {"length_mm":{"base":summarize(Lb),"cgtad":summarize(Lc)},
        "curv_1_per_mm":{"base":summarize(Kb),"cgtad":summarize(Kc)}}

def fmt(x,p=3):
    if not np.isfinite(x): return '---'
    return f"{x:.{p}f}" if (1e-4 <= abs(x) < 1e4) else f"{x:.{p}e}"

os.makedirs("tex", exist_ok=True)
lines = [
  "% Auto-generated: Baseline(u=0) vs CGTAD",
  rf"\providecommand{{\CompLenMeanBase}}{{{fmt(sums['length_mm']['base']['mean'],2)}}}",
  rf"\providecommand{{\CompLenMeanCG}}{{{fmt(sums['length_mm']['cgtad']['mean'],2)}}}",
  rf"\providecommand{{\CompCurvMeanBase}}{{{fmt(sums['curv_1_per_mm']['base']['mean'],3)}}}",
  rf"\providecommand{{\CompCurvMeanCG}}{{{fmt(sums['curv_1_per_mm']['cgtad']['mean'],3)}}}",
]
open("tex/compare_macros_generated.tex","w",encoding="utf-8").write("\n".join(lines)+"\n")
print("[macros] tex/compare_macros_generated.tex")


AssertionError: ベースライン .trk が見つかりません（先に生成を）

In [9]:
# ========== TRK 自動検出 ＆ 比較/単独評価 ワンセル ==========
import os, glob, numpy as np, matplotlib.pyplot as plt
from nibabel.streamlines import load as trk_load

# ---- 1) ユーティリティ（長さ/曲率＆抽出） ----
def _len_mm(s):
    if s.shape[0] < 2: return 0.0
    d = np.diff(s, axis=0); return float(np.linalg.norm(d, axis=1).sum())

def _curv_mean(s, eps=1e-9):
    if s.shape[0] < 3: return np.nan
    p0,p1,p2 = s[:-2], s[1:-1], s[2:]
    v1 = p1 - p0; v2 = p2 - p1
    L1 = np.linalg.norm(v1, axis=1) + eps
    L2 = np.linalg.norm(v2, axis=1) + eps
    ct = np.clip(np.sum(v1*v2, axis=1)/(L1*L2), -1, 1)
    th = np.arccos(ct); smean = 0.5*(L1+L2) + eps
    k = 2.0*np.sin(0.5*th)/smean  # 1/mm
    return float(np.nanmean(k)) if np.isfinite(k).all() else np.nan

def extract_arrays(trk_path):
    obj = trk_load(trk_path)
    Ls, Ks = [], []
    for s in obj.streamlines:
        s = np.asarray(s, dtype=np.float64)
        if s.shape[0] >= 2: Ls.append(_len_mm(s))
        if s.shape[0] >= 3:
            kv = _curv_mean(s)
            if np.isfinite(kv): Ks.append(kv)
    return np.array(Ls, dtype=float), np.array(Ks, dtype=float)

def summarize(arr):
    arr = arr[np.isfinite(arr)]
    return dict(mean=float(np.mean(arr)) if arr.size else float("nan"),
                std =float(np.std(arr))  if arr.size else float("nan"),
                n=int(arr.size))

def fmt(x,p=3):
    if not np.isfinite(x): return '---'
    return f"{x:.{p}f}" if (1e-4 <= abs(x) < 1e4) else f"{x:.{p}e}"

def plot_overlay(a,b,title,xlabel,out,bins=64):
    plt.figure(figsize=(6.4,5))
    if a.size and b.size:
        lo = float(np.nanmin(np.r_[a,b])); hi = float(np.nanmax(np.r_[a,b]))
    elif a.size:
        lo = float(np.nanmin(a)); hi = float(np.nanmax(a))
    elif b.size:
        lo = float(np.nanmin(b)); hi = float(np.nanmax(b))
    else:
        lo, hi = 0.0, 1.0
    lo, hi = max(lo,0.0), max(hi,1e-6)
    edges = np.linspace(lo, hi, bins+1)
    if a.size: plt.hist(a, bins=edges, alpha=0.6, label="Baseline (u=0)")
    if b.size: plt.hist(b, bins=edges, alpha=0.6, label="CGTAD")
    plt.title(title); plt.xlabel(xlabel); plt.ylabel("count")
    if a.size and b.size: plt.legend()
    os.makedirs(os.path.dirname(out), exist_ok=True)
    plt.tight_layout(); plt.savefig(out, bbox_inches="tight"); plt.close()
    print("[saved]", out)

# ---- 2) .trk を自動検出（/content と Drive） ----
cands = sorted(set(
    glob.glob("/content/**/*.trk", recursive=True) +
    glob.glob("/content/drive/**/*.trk", recursive=True)
))
print("trk candidates:"); [print(" -", p) for p in cands]

# ヒューリスティックで拾う
CGTAD_TRK = next((p for p in cands
                  if os.path.basename(p).lower().startswith(("b_stream","cgtad"))), None)
BASE_TRK  = next((p for p in cands
                  if "baseline" in os.path.basename(p).lower()), None)

# 手元のファイル状況に合わせて最後の救済（左ツリーに見えているやつ）
if CGTAD_TRK is None and os.path.exists("/content/B_streamlines.trk"):
    CGTAD_TRK = "/content/B_streamlines.trk"

print("\nPicked:")
print("  CGTAD :", CGTAD_TRK)
print("  BASE  :", BASE_TRK, "\n")

# ---- 3) 評価・出力 ----
os.makedirs("tex", exist_ok=True)

if CGTAD_TRK and BASE_TRK and os.path.exists(CGTAD_TRK) and os.path.exists(BASE_TRK):
    # 比較
    Lb,Kb = extract_arrays(BASE_TRK)
    Lc,Kc = extract_arrays(CGTAD_TRK)
    print("[counts] baseline:", len(Lb), " CGTAD:", len(Lc))
    plot_overlay(Lb,Lc, "Length distribution (Baseline vs CGTAD)", "Length (mm)",
                 "/content/compare_eval/overlay_Length_mm.pdf")
    plot_overlay(Kb,Kc, "Curvature distribution (Baseline vs CGTAD)", "Curvature (1/mm)",
                 "/content/compare_eval/overlay_curvature_1_mm.pdf")

    sums = {"length_mm":{"base":summarize(Lb),"cgtad":summarize(Lc)},
            "curv_1_per_mm":{"base":summarize(Kb),"cgtad":summarize(Kc)}}
    lines = [
      "% Auto-generated: Baseline(u=0) vs CGTAD",
      rf"\providecommand{{\CompLenMeanBase}}{{{fmt(sums['length_mm']['base']['mean'],2)}}}",
      rf"\providecommand{{\CompLenMeanCG}}{{{fmt(sums['length_mm']['cgtad']['mean'],2)}}}",
      rf"\providecommand{{\CompCurvMeanBase}}{{{fmt(sums['curv_1_per_mm']['base']['mean'],3)}}}",
      rf"\providecommand{{\CompCurvMeanCG}}{{{fmt(sums['curv_1_per_mm']['cgtad']['mean'],3)}}}",
    ]
    open("tex/compare_macros_generated.tex","w",encoding="utf-8").write("\n".join(lines)+"\n")
    print("[macros] tex/compare_macros_generated.tex")

elif CGTAD_TRK and os.path.exists(CGTAD_TRK):
    # CGTAD 単独（Baseline 未入手時の暫定）
    Lc,Kc = extract_arrays(CGTAD_TRK)
    print("[counts] CGTAD:", len(Lc))
    plot_overlay(np.array([]),Lc, "CGTAD: Length distribution", "Length (mm)",
                 "/content/compare_eval/cgtad_Length_mm.pdf")
    plot_overlay(np.array([]),Kc, "CGTAD: Curvature distribution", "Curvature (1/mm)",
                 "/content/compare_eval/cgtad_curvature_1_mm.pdf")

    sums = {"length_mm":{"cgtad":summarize(Lc)},
            "curv_1_per_mm":{"cgtad":summarize(Kc)}}
    lines = [
      "% Auto-generated: CGTAD only",
      rf"\providecommand{{\CGOnlyLenMean}}{{{fmt(sums['length_mm']['cgtad']['mean'],2)}}}",
      rf"\providecommand{{\CGOnlyCurvMean}}{{{fmt(sums['curv_1_per_mm']['cgtad']['mean'],3)}}}",
    ]
    open("tex/cgtad_only_macros_generated.tex","w",encoding="utf-8").write("\n".join(lines)+"\n")
    print("[macros] tex/cgtad_only_macros_generated.tex")
    print("\n※ Baseline(u=0) が無いので比較はスキップ。後で u=0 を作れば重ね図も出せます。")
else:
    raise SystemExit("❌ .trk が見つかりません。左ツリーの位置を確認して下さい。")
# ==========================================================


trk candidates:
 - /content/B_streamlines.trk
 - /content/drive/MyDrive/CGTAD/B_streamlines.trk

Picked:
  CGTAD : /content/B_streamlines.trk
  BASE  : None 

[counts] CGTAD: 197959
[saved] /content/compare_eval/cgtad_Length_mm.pdf
[saved] /content/compare_eval/cgtad_curvature_1_mm.pdf
[macros] tex/cgtad_only_macros_generated.tex

※ Baseline(u=0) が無いので比較はスキップ。後で u=0 を作れば重ね図も出せます。


In [10]:
# === Stanford HARDI を取得して Baseline(u=0) 生成 → 既存の比較セルが拾える場所に保存 ===
import os, numpy as np
from dipy.data import get_fnames
from dipy.io.image import load_nifti, save_nifti
from dipy.core.gradients import gradient_table
from dipy.segment.mask import median_otsu
from dipy.reconst.csdeconv import auto_response_ssst, ConstrainedSphericalDeconvModel
from dipy.direction import peaks_from_model
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.stopping_criterion import BinaryStoppingCriterion
from dipy.tracking.streamline import Streamlines
from dipy.io.streamline import save_trk

# 1) データ取得（約 ~100MB）
fraw, fbval, fbvec = get_fnames('stanford_hardi')

# 2) 前処理→CSD→追跡（u=0: 制御なし）
data, affine = load_nifti(fraw)
bvals = np.loadtxt(fbval); bvecs = np.loadtxt(fbvec)
gtab  = gradient_table(bvals, bvecs)

maskdata, mask = median_otsu(data, vol_idx=np.where(bvals>50)[0], numpass=2)
response, ratio = auto_response_ssst(gtab, maskdata, 10, 0.7)
csd = ConstrainedSphericalDeconvModel(gtab, response)
peaks = peaks_from_model(model=csd, data=maskdata, sphere=None,
                         relative_peak_threshold=0.5, min_separation_angle=25,
                         mask=mask, return_sh=False, normalize_peaks=True)

stop = BinaryStoppingCriterion(mask)
seeds = np.argwhere(mask)
STEP_MM = 0.5
streamlines = Streamlines(LocalTracking(peaks, stop, seeds, affine,
                                        step_size=STEP_MM, max_cross=1, return_all=False))

os.makedirs("/content", exist_ok=True)
BASE_TRK = "/content/baseline_u0.trk"
save_trk(Streamlines(streamlines), BASE_TRK, affine=affine, shape=data.shape[:3])
print("[saved baseline]", BASE_TRK, "n=", len(streamlines))


  0%|          | 0/5578 [00:00<?, ? MB/s]

  0%|          | 0/1 [00:00<?, ? MB/s]

  0%|          | 0/1 [00:00<?, ? MB/s]

TypeError: auto_response_ssst() got an unexpected keyword argument 'roi_radius'

In [11]:
# === 修正一発セル：auto_response_ssst と peaks_from_model を互換呼び出しに統一 ===
# 既に data/mask/maskdata/gtab/affine が定義済みならそれを利用。
# 未定義なら Stanford HARDI を自動取得して実行します。

import numpy as np

# 必要モジュール
from dipy.reconst.csdeconv import auto_response_ssst, ConstrainedSphericalDeconvModel
from dipy.direction import peaks_from_model
from dipy.data import default_sphere
from dipy.tracking import utils
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.stopping_criterion import BinaryStoppingCriterion
from dipy.tracking.streamline import Streamlines
from dipy.io.streamline import save_trk

# ---- 未定義ならデータ読み込み（Stanford HARDI）----
try:
    data, mask, maskdata, gtab, affine  # 参照して存在チェック
except NameError:
    from dipy.data import get_fnames
    from dipy.io.image import load_nifti
    from dipy.segment.mask import median_otsu
    from dipy.core.gradients import gradient_table

    dwi_f, bval_f, bvec_f = get_fnames('stanford_hardi')
    data, affine = load_nifti(dwi_f)
    bvals = np.loadtxt(bval_f)
    bvecs_raw = np.loadtxt(bvec_f)
    bvecs = bvecs_raw.T if bvecs_raw.shape[0] == 3 else bvecs_raw
    gtab = gradient_table(bvals, bvecs)

    b0_idx = np.where(bvals < 50)[0]
    maskdata, mask = median_otsu(data, vol_idx=b0_idx, numpass=2)

# ---- 1) auto_response_ssst（古い dipy はキーワード不可なので位置引数で）----
#    デフォルト相当: roi_radius=10, fa_thr=0.7
response, ratio = auto_response_ssst(gtab, maskdata, 10, 0.7)

# ---- 2) CSD モデル ----
csd = ConstrainedSphericalDeconvModel(gtab, response)

# ---- 3) peaks_from_model：sphere を default_sphere に明示 ----
peaks = peaks_from_model(
    model=csd,
    data=maskdata,
    sphere=default_sphere,
    relative_peak_threshold=0.5,
    min_separation_angle=25,
    mask=mask,
    return_sh=False,
    normalize_peaks=True,
)

# ---- 4) u=0 のトラッキング（制御なし）----
STEP_MM = 0.5
seeds = utils.seeds_from_mask(mask, density=1, affine=affine)
stop = BinaryStoppingCriterion(mask)
streamlines = Streamlines(LocalTracking(peaks, stop, seeds, affine, step_size=STEP_MM))

# ---- 5) 保存 ----
out_trk = "/content/baseline_u0.trk"
save_trk(streamlines, out_trk, affine=affine, shape=data.shape[:3])
print(f"[saved] {out_trk}  | n={len(streamlines):,}")


TypeError: len() of unsized object

In [12]:
# === 修正一発セル：u=0 Baseline 生成（ROI半径を3タプルに） ===
import numpy as np
from dipy.reconst.csdeconv import auto_response_ssst, ConstrainedSphericalDeconvModel
from dipy.direction import peaks_from_model
from dipy.data import default_sphere, get_fnames
from dipy.io.image import load_nifti
from dipy.segment.mask import median_otsu
from dipy.core.gradients import gradient_table
from dipy.tracking import utils
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.stopping_criterion import BinaryStoppingCriterion
from dipy.tracking.streamline import Streamlines
from dipy.io.streamline import save_trk

# ---- データ未定義なら Stanford HARDI を取得 ----
try:
    data, mask, maskdata, gtab, affine  # 存在チェック
except NameError:
    dwi_f, bval_f, bvec_f = get_fnames('stanford_hardi')
    data, affine = load_nifti(dwi_f)
    bvals = np.loadtxt(bval_f)
    bvecs_raw = np.loadtxt(bvec_f)
    bvecs = bvecs_raw.T if bvecs_raw.shape[0] == 3 else bvecs_raw
    gtab = gradient_table(bvals, bvecs)
    b0_idx = np.where(bvals < 50)[0]
    maskdata, mask = median_otsu(data, vol_idx=b0_idx, numpass=2)

# ---- 1) auto_response_ssst：ROI半径を (10,10,10) に ----
response, ratio = auto_response_ssst(gtab, maskdata, (10, 10, 10), 0.7)

# ---- 2) CSD モデル ----
csd = ConstrainedSphericalDeconvModel(gtab, response)

# ---- 3) Peaks 抽出（sphere を明示）----
peaks = peaks_from_model(
    model=csd,
    data=maskdata,
    sphere=default_sphere,
    relative_peak_threshold=0.5,
    min_separation_angle=25,
    mask=mask,
    return_sh=False,
    normalize_peaks=True,
)

# ---- 4) u=0 トラッキング ----
STEP_MM = 0.5
seeds = utils.seeds_from_mask(mask, density=1, affine=affine)
stop = BinaryStoppingCriterion(mask)
streamlines = Streamlines(LocalTracking(peaks, stop, seeds, affine, step_size=STEP_MM))

# ---- 5) 保存 ----
out_trk = "/content/baseline_u0.trk"
save_trk(streamlines, out_trk, affine=affine, shape=data.shape[:3])
print(f"[saved] {out_trk}  | n={len(streamlines):,}")


/usr/local/lib/python3.11/dist-packages/dipy/reconst/csdeconv.py:954: UserWarning: No voxel with a FA higher than 0.7 were found.
        Try a larger roi or a lower threshold.
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/dipy/reconst/csdeconv.py:1009: UserWarning: No voxel in mask with value > 0 were found.
  warnings.warn(msg, UserWarning)


ValueError: Input must be 1- or 2-d.

In [13]:
# === u=0 Baseline 生成：auto_response が失敗しても自動でフォールバックする完成版 ===
import numpy as np
from dipy.io.image import load_nifti
from dipy.core.gradients import gradient_table
from dipy.segment.mask import median_otsu
from dipy.data import default_sphere, get_fnames
from dipy.reconst.csdeconv import auto_response_ssst, ConstrainedSphericalDeconvModel
from dipy.reconst.dti import TensorModel
from dipy.direction import peaks_from_model
from dipy.tracking import utils
from dipy.tracking.stopping_criterion import BinaryStoppingCriterion
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.streamline import Streamlines
from dipy.io.streamline import save_trk

# --- 任意で自前データに置き換え可能（None のままなら stanford_hardi を使用） ---
DWI_PATH = None  # 例: "/content/drive/MyDrive/xxx/dwi.nii.gz"
BVAL_PATH = None # 例: "/content/drive/MyDrive/xxx/bvals"
BVEC_PATH = None # 例: "/content/drive/MyDrive/xxx/bvecs"

def load_dataset():
    if DWI_PATH and BVAL_PATH and BVEC_PATH:
        data, affine = load_nifti(DWI_PATH)
        bvals = np.loadtxt(BVAL_PATH)
        bvecs_raw = np.loadtxt(BVEC_PATH)
        bvecs = bvecs_raw.T if bvecs_raw.shape[0] == 3 else bvecs_raw
        gtab = gradient_table(bvals, bvecs)
    else:
        dwi_f, bval_f, bvec_f = get_fnames('stanford_hardi')
        data, affine = load_nifti(dwi_f)
        bvals = np.loadtxt(bval_f)
        bvecs_raw = np.loadtxt(bvec_f)
        bvecs = bvecs_raw.T if bvecs_raw.shape[0] == 3 else bvecs_raw
        gtab = gradient_table(bvals, bvecs)
    return data, affine, gtab, bvals

def center_roi_mask(mask, radius):
    """マスクの外接ボックス中心付近の立方体ROIを返す"""
    inds = np.array(np.where(mask))
    if inds.size == 0:
        return np.zeros_like(mask, dtype=bool)
    mins = inds.min(axis=1)
    maxs = inds.max(axis=1)
    ctr  = ((mins + maxs) // 2).astype(int)
    xs = [max(0, ctr[0]-radius), min(mask.shape[0], ctr[0]+radius+1)]
    ys = [max(0, ctr[1]-radius), min(mask.shape[1], ctr[1]+radius+1)]
    zs = [max(0, ctr[2]-radius), min(mask.shape[2], ctr[2]+radius+1)]
    roi = np.zeros_like(mask, dtype=bool)
    roi[xs[0]:xs[1], ys[0]:ys[1], zs[0]:zs[1]] = True
    return roi & mask

def robust_response(gtab, data, mask):
    """
    まず auto_response_ssst を試し、ダメなら DTI を小ROIでフィットして
    response(evals, S0) を推定して返す。
    """
    # 1) auto_response_ssst を閾値/半径を変えながら試行
    for fa_thr in (0.7, 0.6, 0.5, 0.4, 0.3):
        for rad in ((10,10,10), (12,12,12), (15,15,15)):
            try:
                resp, ratio = auto_response_ssst(gtab, data, rad, fa_thr)
                # DIPY では resp が Response オブジェクト or (evals,S0) 互換
                evals = getattr(resp, "evals", None) or resp[0]
                S0    = getattr(resp, "S0",    None) or resp[1]
                evals = np.asarray(evals).ravel()
                if evals.size == 3 and np.all(np.isfinite(evals)) and np.isfinite(S0) and S0 > 0:
                    print(f"[auto_response] OK  fa_thr={fa_thr}, roi={rad}, evals={evals}, S0={S0:.1f}")
                    return (evals, S0)
            except Exception as e:
                # 次の設定で再試行
                continue

    # 2) フォールバック：DTI を ROI でフィットし median を使う
    b0 = gtab.bvals < 50
    S0_vol = data[..., b0].mean(-1)
    roi = center_roi_mask(mask, radius=12)
    if roi.sum() < 100:
        roi = center_roi_mask(mask, radius=16)
    if roi.sum() < 50:
        # 最終手段：全マスク
        roi = mask

    ten = TensorModel(gtab).fit(data, mask=roi)
    evals_roi = np.asarray(ten.evals)[roi]  # (N,3)
    if evals_roi.size == 0:
        # 典型的なWM値にフォールバック（mm^2/s）
        evals_med = np.array([1.7e-3, 3.0e-4, 3.0e-4])
    else:
        evals_med = np.nanmedian(evals_roi, axis=0)
    S0_med = float(np.nanmedian(S0_vol[roi])) if roi.any() else float(np.nanmedian(S0_vol))

    print(f"[fallback DTI] evals={evals_med}, S0={S0_med:.1f}, roi_vox={int(roi.sum())}")
    return (evals_med, S0_med)

# ---- パイプライン ----
data, affine, gtab, bvals = load_dataset()
b0_idx = np.where(bvals < 50)[0]
maskdata, mask = median_otsu(data, vol_idx=b0_idx, numpass=2)

response = robust_response(gtab, maskdata, mask)
csd = ConstrainedSphericalDeconvModel(gtab, response)

peaks = peaks_from_model(
    model=csd,
    data=maskdata,
    sphere=default_sphere,
    relative_peak_threshold=0.5,
    min_separation_angle=25,
    mask=mask,
    return_sh=False,
    normalize_peaks=True,
)

STEP_MM = 0.5
seeds = utils.seeds_from_mask(mask, density=1, affine=affine)
stop = BinaryStoppingCriterion(mask)
streamlines = Streamlines(LocalTracking(peaks, stop, seeds, affine, step_size=STEP_MM))

out_trk = "/content/baseline_u0.trk"
save_trk(streamlines, out_trk, affine=affine, shape=data.shape[:3])
print(f"[saved] {out_trk} | n={len(streamlines):,}")


[fallback DTI] evals=[0.00111117 0.00066572 0.00053278], S0=471.8, roi_vox=14482


TypeError: save_generator.<locals>.f_gen() got an unexpected keyword argument 'affine'

In [14]:
# ===== QC for /content/baseline.trk =====
import os, numpy as np
from dipy.io.streamline import load_trk
from dipy.io.stateful_tractogram import Space
from dipy.tracking.streamline import length
import matplotlib.pyplot as plt

trk = "/content/baseline.trk"
if not os.path.exists(trk):
    raise FileNotFoundError(f"not found: {trk}")

sft = load_trk(trk, "same")
sft.to_space(Space.RASMM)  # 長さを mm で評価
sl = sft.streamlines
n = len(sl)
lens = np.array([length(s) for s in sl]) if n else np.array([])

print(f"[OK] streamlines: {n}")
if n:
    print(f" length (mm): mean={lens.mean():.1f} ± {lens.std():.1f}  "
          f"min/max={lens.min():.1f}/{lens.max():.1f}")

    plt.figure()
    plt.hist(lens, bins=30)
    plt.xlabel("Streamline length (mm)")
    plt.ylabel("Count")
    plt.title("Length distribution")
    plt.tight_layout()
    plt.savefig("/content/len_hist.png", dpi=200)
    plt.close()
    print("[saved] /content/len_hist.png")


FileNotFoundError: not found: /content/baseline.trk

In [15]:
# --- Save streamlines correctly on DIPY 1.8 ---
import numpy as np, nibabel as nib
from dipy.io.stateful_tractogram import StatefulTractogram, Space
from dipy.io.streamline import save_trk

out_trk = "/content/baseline.trk"

# 参照画像（空ボリューム）を作成
ref_img = nib.Nifti1Image(np.zeros(data.shape[:3], dtype=np.float32), affine)

# LocalTracking の出力は voxel 座標なので Space.VOX を指定
sft = StatefulTractogram(streamlines, ref_img, Space.VOX)
save_trk(sft, out_trk)

print("[saved]", out_trk)


ERROR:StatefulTractogram:Voxel space values lower than 0.0.
ERROR:StatefulTractogram:Voxel space values higher than dimensions.


ValueError: Bounding box is not valid in voxel space, cannot load a valid file if some coordinates are invalid.
Please set bbox_valid_check to False and then use the function remove_invalid_streamlines to discard invalid streamlines.

In [20]:
# --- QC: load whichever exists and print stats ---
import os, numpy as np, matplotlib.pyplot as plt
from dipy.io.streamline import load_trk
from dipy.io.stateful_tractogram import Space
from dipy.tracking.streamline import length

candidates = ["/content/baseline.trk", "/content/B_streamlines.trk"]
trk = next((p for p in candidates if os.path.exists(p)), None)
if trk is None:
    raise FileNotFoundError(f"not found: {candidates}")

sft = load_trk(trk, "same")
sft.to_space(Space.RASMM)  # 長さを mm で評価
sl = sft.streamlines
lens = np.array([length(s) for s in sl]) if len(sl) else np.array([])

print(f"[OK] file: {trk}")
print(f"streamlines: {len(sl)}")
if len(sl):
    print(f"length (mm): mean={lens.mean():.1f} ± {lens.std():.1f}  "
          f"min/max={lens.min():.1f}/{lens.max():.1f}")
    plt.figure()
    plt.hist(lens, bins=30)
    plt.xlabel("Streamline length (mm)"); plt.ylabel("Count")
    plt.title("Length distribution"); plt.tight_layout()
    plt.savefig("/content/len_hist.png", dpi=200); plt.close()
    print("[saved] /content/len_hist.png")


[OK] file: /content/B_streamlines.trk
streamlines: 204419
length (mm): mean=14.7 ± 25.0  min/max=0.0/253.5
[saved] /content/len_hist.png


In [21]:
# === Streamline length ヒストグラムをPDF出力 ===
import os, numpy as np, matplotlib.pyplot as plt
from dipy.io.streamline import load_trk
from dipy.io.stateful_tractogram import Space
from dipy.tracking.streamline import length

# フォントをアウトライン化せずTrueType埋め込み（Illustrator等で編集しやすい）
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["ps.fonttype"]  = 42

candidates = ["/content/baseline.trk", "/content/B_streamlines.trk"]
trk = next((p for p in candidates if os.path.exists(p)), None)
if trk is None:
    raise FileNotFoundError(f"not found: {candidates}")

sft = load_trk(trk, "same")
sft.to_space(Space.RASMM)  # 長さをmmで評価
sl   = sft.streamlines
lens = np.array([length(s) for s in sl]) if len(sl) else np.array([])

print(f"[OK] file: {trk}")
print(f"streamlines: {len(sl)}")
if len(sl):
    print(f"length (mm): mean={lens.mean():.1f} ± {lens.std():.1f}  "
          f"min/max={lens.min():.1f}/{lens.max():.1f}")

    # 図を再描画 → PDF/PNG で保存
    plt.figure(figsize=(6,4))
    plt.hist(lens, bins=30)
    plt.xlabel("Streamline length (mm)")
    plt.ylabel("Count")
    plt.title("Length distribution")
    plt.tight_layout()
    plt.savefig("/content/len_hist.pdf")            # ベクター（推奨）
    plt.savefig("/content/len_hist.png", dpi=300)   # 参考に高解像度PNGも
    plt.close()
    print("[saved] /content/len_hist.pdf")
    print("[saved] /content/len_hist.png")


[OK] file: /content/B_streamlines.trk
streamlines: 204419
length (mm): mean=14.7 ± 25.0  min/max=0.0/253.5
[saved] /content/len_hist.pdf
[saved] /content/len_hist.png


In [27]:
# === ECDF comparison of streamline lengths across methods (PDF) ===
import os, numpy as np, matplotlib.pyplot as plt
from dipy.io.streamline import load_trk
from dipy.io.stateful_tractogram import Space
from dipy.tracking.streamline import length
from scipy.stats import ks_2samp

plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["ps.fonttype"]  = 42

# ここに比較したい .trk を列挙（存在するものだけ拾います）
candidates = {
    "Baseline (CSD-det)": "/content/baseline.trk",
    "Proposed (CGTAD)":   "/content/B_streamlines.trk",
    # "SOTA: UKF": "/content/ukf.trk",
    # "SOTA: Probtrackx": "/content/prob.trk",
}

lengths = {}
for name, path in candidates.items():
    if not os.path.exists(path):
        continue
    sft = load_trk(path, "same", bbox_valid_check=False)
    sft.to_space(Space.RASMM)
    L = np.array([length(s) for s in sft.streamlines], dtype=float)
    L = L[np.isfinite(L)]
    lengths[name] = L

# 要約表示 & KS検定（Baselineを基準に）
keys = list(lengths.keys())
if not keys:
    raise FileNotFoundError("No .trk files found.")
base = keys[0]
print(f"[base] {base}  n={len(lengths[base])}")

for k in keys:
    L = lengths[k]
    if len(L)==0: continue
    q5, q50, q95 = np.percentile(L, [5,50,95])
    print(f"[{k:>18}] n={len(L):5d}  mean={L.mean():6.1f}  sd={L.std():5.1f}  "
          f"p5/p50/p95={q5:5.1f}/{q50:5.1f}/{q95:5.1f}")
    if k != base and len(lengths[base]) and len(L):
        D, p = ks_2samp(lengths[base], L)
        print(f"  KS vs {base}: D={D:.3f}, p={p:.3e}")

# ECDF 図（PDF保存）
plt.figure(figsize=(6,4))
for k, L in lengths.items():
    if len(L)==0: continue
    x = np.sort(L)
    y = np.arange(1, len(x)+1)/len(x)
    plt.step(x, y, where="post", label=k)
plt.xlabel("Streamline length (mm)")
plt.ylabel("ECDF")
plt.xlim(left=0)
plt.legend()
plt.tight_layout()
plt.savefig("/content/len_ecdf.pdf")
plt.close()
print("[saved] /content/len_ecdf.pdf")


[base] Baseline (CSD-det)  n=14829
[Baseline (CSD-det)] n=14829  mean=  94.8  sd= 89.4  p5/p50/p95=  3.0/ 69.0/276.0
[  Proposed (CGTAD)] n=204419  mean=  14.7  sd= 25.0  p5/p50/p95=  2.0/  5.0/ 77.5
  KS vs Baseline (CSD-det): D=0.612, p=0.000e+00
[saved] /content/len_ecdf.pdf


In [28]:
# === Bundle overlap / Dice coefficient ===
import os, numpy as np, nibabel as nib
from dipy.io.streamline import load_trk
from dipy.io.stateful_tractogram import Space

# ---- 入力（必要に応じて差し替え）----
paths = {
    "Baseline": "/content/baseline.trk",
    "Proposed": "/content/B_streamlines.trk",
    # "SOTA-UKF": "/content/ukf.trk",
}
roi_mask_path = None  # 例: "/content/ROI_bundle.nii.gz"  (ROIがあれば指定; 無ければ None)

# ---- ユーティリティ ----
def to_voxel_streamlines(trk_path):
    sft = load_trk(trk_path, "same", bbox_valid_check=False)
    sft.to_space(Space.VOX)
    return sft.streamlines

def infer_shape_from_streamlines(streamlines_list, margin=2):
    pts = np.vstack([p for sl in streamlines_list for p in sl])  # すべての点を結合
    mx = np.ceil(pts.max(axis=0) + margin).astype(int)
    mx = np.maximum(mx, 1)
    return tuple(mx.tolist())

def visitation_mask(streamlines, vol_shape):
    m = np.zeros(vol_shape, dtype=bool)
    for sl in streamlines:
        ijk = np.rint(sl).astype(int)
        ijk = ijk[(ijk[:,0]>=0)&(ijk[:,1]>=0)&(ijk[:,2]>=0)]
        ijk[:,0] = np.clip(ijk[:,0], 0, vol_shape[0]-1)
        ijk[:,1] = np.clip(ijk[:,1], 0, vol_shape[1]-1)
        ijk[:,2] = np.clip(ijk[:,2], 0, vol_shape[2]-1)
        m[ijk[:,0], ijk[:,1], ijk[:,2]] = True
    return m

def dice_coef(A, B, eps=1e-8):
    inter = np.logical_and(A, B).sum()
    return 2.0 * inter / (A.sum() + B.sum() + eps)

# ---- 読み込み＆体積マスク化 ----
name2sl = {}
for name, p in paths.items():
    if os.path.exists(p):
        name2sl[name] = to_voxel_streamlines(p)
    else:
        print(f"[skip] not found: {p}")
        continue

if not name2sl:
    raise FileNotFoundError("No .trk found.")

# 体積サイズを決める（ROIがあればそれに合わせる）
if roi_mask_path and os.path.exists(roi_mask_path):
    roi_img = nib.load(roi_mask_path)
    vol_shape = roi_img.shape[:3]
    roi = roi_img.get_fdata() > 0
else:
    vol_shape = infer_shape_from_streamlines(list(name2sl.values()))
    roi = None

name2mask = {name: visitation_mask(sl, vol_shape) for name, sl in name2sl.items()}
if roi is not None:
    name2mask = {k: np.logical_and(v, roi) for k, v in name2mask.items()}

# ---- Dice の一覧（1つ目を基準に）----
names = list(name2mask.keys())
base = names[0]
print(f"[base for Dice] {base}")
for n in names:
    if n == base:
        continue
    d = dice_coef(name2mask[base], name2mask[n])
    print(f"Dice({base} vs {n}) = {d:.3f}")

# もしGTのバンドルマスク(roi_mask_path)が "正解" なら、そのDiceも出す：
if roi is not None:
    for n in names:
        d_gt = dice_coef(name2mask[n], roi)
        print(f"Dice({n} vs ROI-GT) = {d_gt:.3f}")


[base for Dice] Baseline
Dice(Baseline vs Proposed) = 0.310


In [30]:
# === Crossing-angle error (voxel-wise, k=2 dominant undirected orientations) ===
import os, numpy as np, nibabel as nib
from dipy.io.streamline import load_trk
from dipy.io.stateful_tractogram import Space

# ---- 入力 ----
# 比較したい手法（最低2つ）。最初を基準(Reference)にして pairwise 差も出せます。
paths = {
    "Baseline": "/content/baseline.trk",
    "Proposed": "/content/B_streamlines.trk",
    # "SOTA-UKF": "/content/ukf.trk",
}

roi_mask_path = None  # ROIを使うならパスを指定


# 既知のGT交差角（度）を各ボクセルに持つnii.gz（なければ None）
# 例：一定角(60°)でよければ scalar_gt_angle_deg に値を入れる
gt_angle_map_path = None  # e.g., "/content/gt_crossing_angle_deg.nii.gz"
scalar_gt_angle_deg = None  # e.g., 60.0

# クロッシングボクセルの抽出条件
min_dirs_per_voxel = 12   # そのボクセルに集まる方向ベクトルの最小数
min_separation_deg = 15.0 # 2クラスタの角度がこの値以上なら「交差」とみなす

# ---- 方向ベクトルを集計（voxel空間） ----
def to_voxel_streamlines(trk_path):
    sft = load_trk(trk_path, "same", bbox_valid_check=False)
    sft.to_space(Space.VOX)
    return sft.streamlines

def unit(v):
    n = np.linalg.norm(v)
    return v / (n + 1e-12)

def voxel_orientations(streamlines, vol_shape):
    # 各ボクセルに、ストリームライン区間の方向ベクトル（符号同一視）を貯める
    vox2vecs = {}
    for sl in streamlines:
        if len(sl) < 2:
            continue
        segs = sl[1:] - sl[:-1]              # 区間ベクトル
        mids = (sl[1:] + sl[:-1]) * 0.5      # 中点（どのボクセルに属すか）
        ijk = np.rint(mids).astype(int)
        # 体積外を除外＆クリップ
        valid = (ijk[:,0]>=0)&(ijk[:,1]>=0)&(ijk[:,2]>=0)& \
                (ijk[:,0]<vol_shape[0])&(ijk[:,1]<vol_shape[1])&(ijk[:,2]<vol_shape[2])
        ijk, segs = ijk[valid], segs[valid]
        for v, (i,j,k) in zip(segs, ijk):
            v = unit(v)
            # 向き同一視（-v を v に合わせる）
            if v[0]<0:
                v = -v
            key = (i,j,k)
            if key not in vox2vecs:
                vox2vecs[key] = []
            vox2vecs[key].append(v)
    # numpy化
    for k in list(vox2vecs.keys()):
        vox2vecs[k] = np.asarray(vox2vecs[k], dtype=float)
    return vox2vecs

def two_undirected_means(vecs, iters=12):
    # v ~ -v を同一視した 2-means
    V = vecs.copy()
    if len(V) < 2:
        return None
    # 初期値：1つ目は平均方向、2つ目は最遠方向
    c1 = unit(V.mean(axis=0))
    proj = np.abs(V @ c1)
    c2 = unit(V[np.argmin(proj)])
    for _ in range(iters):
        A, B = [], []
        for v in V:
            d1 = 1 - np.abs(v @ c1)
            d2 = 1 - np.abs(v @ c2)
            if d1 <= d2:
                A.append(v if v @ c1 >= 0 else -v)
            else:
                B.append(v if v @ c2 >= 0 else -v)
        if len(A)==0 or len(B)==0:
            return None
        c1 = unit(np.mean(A, axis=0))
        c2 = unit(np.mean(B, axis=0))
    # 角度は 0–90° に正規化
    ang = np.degrees(np.arccos(np.clip(np.dot(c1, c2), -1, 1)))
    ang = ang if ang <= 90 else 180 - ang
    return ang, c1, c2

def infer_shape_union(paths):
    # すべてのtrkの点域から最小包囲ボリュームを作る
    from itertools import chain
    all_pts = []
    for p in paths:
        sl = to_voxel_streamlines(p)
        if len(sl):
            all_pts.extend(chain.from_iterable(sl))
    P = np.asarray(all_pts) if all_pts else np.zeros((1,3))
    mx = np.ceil(P.max(axis=0) + 2).astype(int)
    mx = np.maximum(mx, 1)
    return tuple(mx.tolist())

# ---- 準備 ----
existing = {n:p for n,p in paths.items() if os.path.exists(p)}
if len(existing) < 1:
    raise FileNotFoundError("No .trk found.")
vol_shape = infer_shape_union(list(existing.values()))

name2angles = {}   # 各手法の {voxel_key: angle_deg}
name2counts = {}   # 各手法の {voxel_key: num_vectors}

for name, p in existing.items():
    sl = to_voxel_streamlines(p)
    vox2vecs = voxel_orientations(sl, vol_shape)
    angs = {}
    cnts = {}
    for key, V in vox2vecs.items():
        if len(V) < min_dirs_per_voxel:
            continue
        res = two_undirected_means(V)
        if res is None:
            continue
        angle_deg, _, _ = res
        if angle_deg >= min_separation_deg:
            angs[key] = angle_deg
            cnts[key] = len(V)
    name2angles[name] = angs
    name2counts[name] = cnts
    print(f"[{name}] crossing voxels: {len(angs)}")

# ---- 誤差評価（GTあり or 参照法あり）----
def summarize_errors(errs, label):
    if len(errs)==0:
        print(f"{label}: no overlapping crossing voxels.")
        return
    E = np.array(errs)
    q = np.percentile(E, [5,50,95])
    print(f"{label}: n={len(E)}  mean={E.mean():.2f}  sd={E.std():.2f}  "
          f"p5/p50/p95={q[0]:.2f}/{q[1]:.2f}/{q[2]:.2f}")

errors = {}

# ケース1: GT角マップ
if gt_angle_map_path and os.path.exists(gt_angle_map_path):
    gt_img = nib.load(gt_angle_map_path); GT = gt_img.get_fdata()
    for name, angs in name2angles.items():
        err = []
        for (i,j,k), a in angs.items():
            if 0<=i<GT.shape[0] and 0<=j<GT.shape[1] and 0<=k<GT.shape[2]:
                g = GT[i,j,k]
                if np.isfinite(g) and g>0:
                    err.append(abs(a - g))
        errors[name] = err
        summarize_errors(err, f"[GT] {name}")

# ケース2: スカラーGT角
elif scalar_gt_angle_deg is not None:
    g = float(scalar_gt_angle_deg)
    for name, angs in name2angles.items():
        err = [abs(a - g) for a in angs.values()]
        errors[name] = err
        summarize_errors(err, f"[GT={g:.1f}°] {name}")

# ケース3: 参照法（最初の手法を基準に pairwise 差）
else:
    names = list(name2angles.keys())
    ref = names[0]
    for name in names[1:]:
        common = set(name2angles[ref].keys()) & set(name2angles[name].keys())
        err = [abs(name2angles[name][k] - name2angles[ref][k]) for k in common]
        errors[(ref, name)] = err
        summarize_errors(err, f"[pairwise] {ref} vs {name}")


KeyboardInterrupt: 

In [25]:
# === Baseline (CSD-deterministic) tractography → /content/baseline.trk ===
import numpy as np, nibabel as nib
from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel, auto_response_ssst
from dipy.direction import peaks_from_model
from dipy.data import default_sphere
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.stopping_criterion import BinaryStoppingCriterion
from dipy.tracking.utils import seeds_from_mask
from dipy.io.stateful_tractogram import StatefulTractogram, Space
from dipy.io.streamline import save_trk

# 1) response（FA=0.7で見つからない環境があるので 0.6 に）
try:
    response, ratio = auto_response_ssst(gtab, data, roi_radius=10, fa_thr=0.6)
except Exception as e:
    # フォールバック：DTIから中央値レスポンスを作る
    from dipy.reconst.dti import TensorModel
    ten = TensorModel(gtab).fit(data, mask=mask)
    evals = np.clip(ten.evals, 1e-4, 3e-3)
    S0 = float(np.median(data[mask].mean(axis=-1)))
    evals_med = np.median(evals[mask], axis=0)
    response = (evals_med, S0)

# 2) CSD + peaks
csd = ConstrainedSphericalDeconvModel(gtab, response, sh_order=8)
peaks = peaks_from_model(
    model=csd, data=data, sphere=default_sphere,
    relative_peak_threshold=0.5, min_separation_angle=25,
    mask=mask, npeaks=2, normalize_peaks=True
)

# 3) seeds / stopping
stop  = BinaryStoppingCriterion(mask)
seeds = seeds_from_mask(mask, density=1, affine=affine)  # mm座標でseed

# 4) LocalTracking（mm座標で出力）
streamlines = list(LocalTracking(
    peaks, stop, seeds, affine=affine,
    step_size=0.5, return_all=False
))

# 5) save（DIPY 1.8 正式手順）
ref_img = nib.Nifti1Image(np.zeros(data.shape[:3], dtype=np.float32), affine)
sft = StatefulTractogram(streamlines, ref_img, Space.RASMM)
save_trk(sft, "/content/baseline.trk")
print("[saved] /content/baseline.trk  | n_streamlines:", len(streamlines))


[saved] /content/baseline.trk  | n_streamlines: 14829


In [26]:
# === quick QC for baseline.trk ===
import os, numpy as np
from dipy.io.streamline import load_trk
from dipy.io.stateful_tractogram import Space
from dipy.tracking.streamline import length
import matplotlib.pyplot as plt

trk = "/content/baseline.trk"
assert os.path.exists(trk), "baseline.trk がありません"

sft = load_trk(trk, "same", bbox_valid_check=False)  # ← 重要
sft.to_space(Space.RASMM)
sl = sft.streamlines
L  = np.array([length(s) for s in sl])
print(f"[OK] baseline streamlines: {len(sl)}  mean={L.mean():.1f} mm")

plt.figure()
plt.hist(L, bins=30); plt.xlabel("length (mm)"); plt.ylabel("count")
plt.tight_layout(); plt.savefig("/content/baseline_len_hist.pdf"); plt.close()
print("[saved] /content/baseline_len_hist.pdf")


[OK] baseline streamlines: 14829  mean=94.8 mm
[saved] /content/baseline_len_hist.pdf


In [32]:
# ===== One-shot SOTA eval (fixed & overwrite) =====
import os, numpy as np, nibabel as nib, matplotlib.pyplot as plt
from dipy.io.streamline import load_trk, save_trk
from dipy.io.stateful_tractogram import StatefulTractogram, Space
from dipy.tracking.streamline import length

# ---------------- Config ----------------
Lmin = 20.0   # mm  以下を除外
tau  = 3      # TDIしきい: 1ボクセルあたりtau本以上を有効とみなす

candidates = {
    "Baseline (CSD-det)": "/content/baseline.trk",
    "Proposed (CGTAD)"  : "/content/B_streamlines.trk",
    # "SOTA: UKF"       : "/content/ukf.trk",
    # "SOTA: Prob"      : "/content/prob.trk",
}

# ---------------- Utilities ----------------
def get_ref_img():
    """参照画像は data/affine を最優先で使用。無ければ最初のtrkから近似。"""
    try:
        ref = nib.Nifti1Image(np.zeros(data.shape[:3], dtype=np.float32), affine)
        return ref
    except Exception:
        # fallback: first existing .trk
        for p in candidates.values():
            if os.path.exists(p):
                sft = load_trk(p, "same", bbox_valid_check=False)
                # dimensions は (X, Y, Z) のvoxel shape相当
                shape = tuple(map(int, sft.dimensions))
                ref = nib.Nifti1Image(np.zeros(shape, dtype=np.float32), sft.affine)
                return ref
        raise RuntimeError("No reference available: set data/affine or provide at least one .trk")

ref_img = get_ref_img()
Ainv    = np.linalg.inv(ref_img.affine)
vol_shape = ref_img.shape[:3]

def in_bounds_mm(sl_mm):
    """mm座標のストリームラインが参照体積内か"""
    P = np.c_[sl_mm, np.ones(len(sl_mm))]
    ijk = (P @ Ainv.T)[:, :3]
    return (
        (ijk[:,0] >= 0).all() and (ijk[:,0] < vol_shape[0]).all() and
        (ijk[:,1] >= 0).all() and (ijk[:,1] < vol_shape[1]).all() and
        (ijk[:,2] >= 0).all() and (ijk[:,2] < vol_shape[2]).all()
    )

def load_to_mm(path):
    sft = load_trk(path, "same", bbox_valid_check=False)
    sft.to_space(Space.RASMM)
    return sft, list(sft.streamlines)

def filter_save_Lmin(path, name):
    """L>=Lmin & in-bounds を適用して _LminXX.trk を上書き保存"""
    sft, sl = load_to_mm(path)
    L = np.array([length(s) for s in sl], dtype=float)
    keep = [s for s, l in zip(sl, L) if np.isfinite(l) and l >= Lmin]
    keep_in = [s for s in keep if in_bounds_mm(s)]
    out_p = path.replace(".trk", f"_Lmin{int(Lmin)}.trk")
    sft_out = StatefulTractogram(keep_in, ref_img, Space.RASMM)
    save_trk(sft_out, out_p, bbox_valid_check=False)
    L_keep = np.array([length(s) for s in keep_in], dtype=float)
    print(f"[saved] {name}: {out_p}  n={len(keep_in)}  "
          f"mean={L_keep.mean():.1f}  p50={np.percentile(L_keep,50):.1f}")
    return out_p, L_keep

def ecdf_pdf(name2L, out_pdf):
    plt.rcParams["pdf.fonttype"] = 42; plt.rcParams["ps.fonttype"] = 42
    plt.figure(figsize=(6,4))
    for name, L in name2L.items():
        if len(L)==0: continue
        x = np.sort(L); y = np.arange(1, len(x)+1)/len(x)
        plt.step(x, y, where="post", label=f"{name} (n={len(L)})")
    plt.xlabel("Streamline length (mm)"); plt.ylabel("ECDF"); plt.xlim(left=Lmin)
    plt.legend(); plt.tight_layout(); plt.savefig(out_pdf); plt.close()
    print(f"[saved] {out_pdf}")

def visitation_mask(sl_vox, vol_shape):
    m = np.zeros(vol_shape, dtype=bool)
    for sl in sl_vox:
        ijk = np.rint(sl).astype(int)
        valid = (ijk[:,0]>=0)&(ijk[:,1]>=0)&(ijk[:,2]>=0)
        ijk = ijk[valid]
        if len(ijk)==0: continue
        ijk[:,0] = np.clip(ijk[:,0], 0, vol_shape[0]-1)
        ijk[:,1] = np.clip(ijk[:,1], 0, vol_shape[1]-1)
        ijk[:,2] = np.clip(ijk[:,2], 0, vol_shape[2]-1)
        m[ijk[:,0], ijk[:,1], ijk[:,2]] = True
    return m

def tdi_counts(sl_vox, vol_shape):
    c = np.zeros(vol_shape, dtype=np.int32)
    for sl in sl_vox:
        ijk = np.rint(sl).astype(int)
        valid = (ijk[:,0]>=0)&(ijk[:,1]>=0)&(ijk[:,2]>=0)
        ijk = ijk[valid]
        if len(ijk)==0: continue
        ijk[:,0] = np.clip(ijk[:,0], 0, vol_shape[0]-1)
        ijk[:,1] = np.clip(ijk[:,1], 0, vol_shape[1]-1)
        ijk[:,2] = np.clip(ijk[:,2], 0, vol_shape[2]-1)
        c[ijk[:,0], ijk[:,1], ijk[:,2]] += 1
    return c

def dice(A,B,eps=1e-8):
    inter = np.logical_and(A,B).sum()
    return 2*inter/(A.sum()+B.sum()+eps)

# ---------------- Run: filter & save ----------------
existing = {n:p for n,p in candidates.items() if os.path.exists(p)}
if not existing:
    raise FileNotFoundError("No .trk found in candidates.")

filtered_paths, name2L = {}, {}
for name, path in existing.items():
    out_p, L_keep = filter_save_Lmin(path, name)
    filtered_paths[name] = out_p
    name2L[name] = L_keep

# ---------------- ECDF (PDF) ----------------
ecdf_pdf(name2L, "/content/len_ecdf_Lmin.pdf")

# ---------------- Dice / TDI-Dice ----------------
# voxel空間へ
name2sl_vox = {}
for name, p in filtered_paths.items():
    sft = load_trk(p, "same", bbox_valid_check=False)
    sft.to_space(Space.VOX)
    name2sl_vox[name] = list(sft.streamlines)

# 体積shapeは参照画像のshapeを採用
vol_shape = ref_img.shape[:3]
name2mask = {n: visitation_mask(sl, vol_shape) for n, sl in name2sl_vox.items()}
names = list(name2mask.keys())
base  = names[0]
for n in names[1:]:
    print(f"Dice(L≥{int(Lmin)}): {base} vs {n} = {dice(name2mask[base], name2mask[n]):.3f}")

name2tdi = {n: tdi_counts(sl, vol_shape) for n, sl in name2sl_vox.items()}
for n in names[1:]:
    A = name2tdi[base] >= tau
    B = name2tdi[n]    >= tau
    print(f"TDI-Dice(τ={tau}, L≥{int(Lmin)}): {base} vs {n} = {dice(A,B):.3f}")

# ---------------- Optional: match N to Baseline ----------------
if len(names) >= 2:
    base_name = names[0]
    for n in names[1:]:
        # 現手法の本数
        sft_b = load_trk(filtered_paths[base_name], "same", bbox_valid_check=False); sft_b.to_space(Space.RASMM)
        sft_p = load_trk(filtered_paths[n],        "same", bbox_valid_check=False); sft_p.to_space(Space.RASMM)
        nb, np_ = len(sft_b.streamlines), len(sft_p.streamlines)
        if np_ > nb and nb > 0:
            rng = np.random.RandomState(0)
            idx = rng.choice(np_, size=nb, replace=False)
            sl_sub = [sft_p.streamlines[i] for i in idx]
            sft_sub = StatefulTractogram(sl_sub, ref_img, Space.RASMM)
            out_p = filtered_paths[n].replace(".trk", "_matchN.trk")
            save_trk(sft_sub, out_p, bbox_valid_check=False)
            print(f"[saved] {n} matched to {base_name}: {out_p}  n={nb}")
        else:
            print(f"[info] {n}: n={np_} ≤ {base_name}: n={nb}  (no downsampling)")


[saved] Baseline (CSD-det): /content/baseline_Lmin20.trk  n=11576  mean=119.0  p50=90.0
[saved] Proposed (CGTAD): /content/B_streamlines_Lmin20.trk  n=36167  mean=59.0  p50=50.5
[saved] /content/len_ecdf_Lmin.pdf
Dice(L≥20): Baseline (CSD-det) vs Proposed (CGTAD) = 0.539
TDI-Dice(τ=3, L≥20): Baseline (CSD-det) vs Proposed (CGTAD) = 0.553
[saved] Proposed (CGTAD) matched to Baseline (CSD-det): /content/B_streamlines_Lmin20_matchN.trk  n=11576


In [33]:
# --- Re-eval with matched-N ---
import os, numpy as np, matplotlib.pyplot as plt
from dipy.io.streamline import load_trk
from dipy.io.stateful_tractogram import Space
from dipy.tracking.streamline import length

base = "/content/baseline_Lmin20.trk"
prop = "/content/B_streamlines_Lmin20_matchN.trk"

def load_len_mm(p):
    sft = load_trk(p, "same", bbox_valid_check=False); sft.to_space(Space.RASMM)
    sl = sft.streamlines
    L  = np.array([length(s) for s in sl], dtype=float)
    return sl, L, sft

sl_b, L_b, sft_b = load_len_mm(base)
sl_p, L_p, sft_p = load_len_mm(prop)

# ECDF（PDF）
plt.rcParams["pdf.fonttype"]=42; plt.rcParams["ps.fonttype"]=42
plt.figure(figsize=(6,4))
for name, L in {"Baseline":L_b, "Proposed (matchN)":L_p}.items():
    x=np.sort(L); y=np.arange(1,len(x)+1)/len(x); plt.step(x,y,where="post",label=f"{name} (n={len(L)})")
plt.xlabel("Streamline length (mm)"); plt.ylabel("ECDF"); plt.tight_layout()
plt.savefig("/content/len_ecdf_Lmin_matchN.pdf"); plt.close()
print("[saved] /content/len_ecdf_Lmin_matchN.pdf")

# Dice（訪問マスク）
import numpy as np
def visitation_mask(sl, shape):
    m=np.zeros(shape,dtype=bool)
    for s in sl:
        ijk=np.rint(s).astype(int)
        ijk[:,0]=np.clip(ijk[:,0],0,shape[0]-1)
        ijk[:,1]=np.clip(ijk[:,1],0,shape[1]-1)
        ijk[:,2]=np.clip(ijk[:,2],0,shape[2]-1)
        m[ijk[:,0],ijk[:,1],ijk[:,2]]=True
    return m
shape = sft_b.dimensions[::-1]  # (X,Y,Z)
A = visitation_mask(sl_b, shape); B = visitation_mask(sl_p, shape)
dice = lambda X,Y: 2*np.logical_and(X,Y).sum()/(X.sum()+Y.sum()+1e-8)
print(f"Dice (matchN, L≥20): {dice(A,B):.3f}")


[saved] /content/len_ecdf_Lmin_matchN.pdf
Dice (matchN, L≥20): 0.478


In [34]:
# === TDI–Dice sensitivity + summary table (PDF+TeX) ===
import os, numpy as np, nibabel as nib, matplotlib.pyplot as plt
from dipy.io.streamline import load_trk
from dipy.io.stateful_tractogram import Space
from dipy.tracking.streamline import length

# ---------------- Config ----------------
# 既に作った Lmin版（と matchN があれば）をここに列挙
base_trk = "/content/baseline_Lmin20.trk"
methods = {
    "Proposed (CGTAD)":         "/content/B_streamlines_Lmin20.trk",
    "Proposed (matchN)":        "/content/B_streamlines_Lmin20_matchN.trk",  # あれば自動採用
    # "SOTA: UKF":              "/content/ukf_Lmin20.trk",
    # "SOTA: Probtrackx":       "/content/prob_Lmin20.trk",
}
tau_list = [1, 3, 5, 10, 15, 20]     # 感度解析する TDI しきい値
roi_mask_path = None                  # 例: "/content/ROI_bundle.nii.gz"（任意）

out_pdf = "/content/tdi_dice_sensitivity.pdf"
out_tex = "/content/table_sota_tdi_dice.tex"

# ---------------- Helpers ----------------
def load_vox(p):
    sft = load_trk(p, "same", bbox_valid_check=False)
    sft.to_space(Space.VOX)
    return list(sft.streamlines), sft

def load_mm(p):
    sft = load_trk(p, "same", bbox_valid_check=False)
    sft.to_space(Space.RASMM)
    sl = list(sft.streamlines)
    L  = np.array([length(s) for s in sl], dtype=float)
    return sl, L, sft

def tdi_counts(sl_vox, vol_shape):
    c = np.zeros(vol_shape, dtype=np.int32)
    for s in sl_vox:
        ijk = np.rint(s).astype(int)
        valid = (ijk[:,0]>=0)&(ijk[:,1]>=0)&(ijk[:,2]>=0)
        ijk = ijk[valid]
        if len(ijk)==0: continue
        ijk[:,0] = np.clip(ijk[:,0], 0, vol_shape[0]-1)
        ijk[:,1] = np.clip(ijk[:,1], 0, vol_shape[1]-1)
        ijk[:,2] = np.clip(ijk[:,2], 0, vol_shape[2]-1)
        c[ijk[:,0], ijk[:,1], ijk[:,2]] += 1
    return c

def dice(A,B,eps=1e-8):
    inter = np.logical_and(A,B).sum()
    return 2*inter/(A.sum()+B.sum()+eps)

# ---------------- Load base + methods ----------------
assert os.path.exists(base_trk), f"not found: {base_trk}"
sl_b_vox, sft_b = load_vox(base_trk)
sl_b_mm,  L_b,  _ = load_mm(base_trk)
vol_shape = sft_b.dimensions[::-1]  # (X,Y,Z)
print(f"[base] n={len(sl_b_mm)}  mean={L_b.mean():.1f}  p50={np.percentile(L_b,50):.1f}")

# ROI（任意）
roi = None
if roi_mask_path and os.path.exists(roi_mask_path):
    roi_img = nib.load(roi_mask_path)
    if roi_img.shape[:3] == vol_shape:
        roi = roi_img.get_fdata() > 0
        print(f"[ROI] applied: {roi_mask_path}")
    else:
        print(f"[warn] ROI shape {roi_img.shape[:3]} != vol {vol_shape} (ignored)")

name2L = {"Baseline": L_b}
name2tdi = {}

# Base TDI
tdi_base = tdi_counts(sl_b_vox, vol_shape)
if roi is not None: tdi_base = tdi_base * roi
name2tdi["Baseline"] = tdi_base

# Methods
existing = {n:p for n,p in methods.items() if os.path.exists(p)}
for name, p in existing.items():
    sl_vox, _ = load_vox(p)
    sl_mm,  L, _  = load_mm(p)
    name2L[name]  = L
    T = tdi_counts(sl_vox, vol_shape)
    if roi is not None: T = T * roi
    name2tdi[name] = T
    print(f"[{name}] n={len(L)}  mean={L.mean():.1f}  p50={np.percentile(L,50):.1f}")

# ---------------- TDI-Dice sensitivity curves ----------------
plt.rcParams["pdf.fonttype"] = 42; plt.rcParams["ps.fonttype"] = 42
plt.figure(figsize=(6,4))
curves = {}  # for table

for name, T in name2tdi.items():
    if name == "Baseline":
        continue
    y = []
    for tau in tau_list:
        A = (name2tdi["Baseline"] >= tau)
        B = (T >= tau)
        y.append(dice(A,B))
    curves[name] = y
    plt.plot(tau_list, y, marker="o", label=name)

plt.xlabel(r"TDI threshold $\tau$  (counts per voxel)")
plt.ylabel("Dice vs Baseline")
plt.xticks(tau_list)
plt.ylim(0,1)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig(out_pdf); plt.close()
print(f"[saved] {out_pdf}")

# ---------------- Build LaTeX table ----------------
# columns: n, mean(mm), p50(mm), Dice@tau=1,3,5,10,15,20（存在するtauのみ）
def fmt(x):
    return "-" if x is None else (f"{x:.3f}" if isinstance(x,float) else str(x))

rows = []
# Baseline row（Dice列は空欄）
rows.append([
    "Baseline",
    len(name2L["Baseline"]),
    float(np.mean(name2L["Baseline"])),
    float(np.percentile(name2L["Baseline"], 50)),
] + ["-"]*len(tau_list))

# Each method
for name in existing.keys():
    L = name2L[name]
    stats = [len(L), float(np.mean(L)), float(np.percentile(L,50))]
    dlist = [float(v) for v in curves[name]]
    rows.append([name] + stats + dlist)

# render TeX
tex_lines = []
tex_lines.append(r"\begin{table}[t]")
tex_lines.append(r"\centering")
cap = r"TDI--Dice sensitivity vs.\ Baseline after length filtering. Dice values are computed on TDI-thresholded visitation maps (voxel counts $\ge \tau$)."
tex_lines.append(r"\caption{" + cap + r"}")
tex_lines.append(r"\label{tab:tdi_dice_sensitivity}")
cols = "l" + "r"*3 + "r"*len(tau_list)
tex_lines.append(r"\begin{tabular}{" + cols + r"}")
tex_lines.append(r"\toprule")
head1 = r"Method & $n$ & mean [mm] & $p_{50}$ [mm]"
head2 = " & " + " & ".join([fr"Dice@$\tau={t}$" for t in tau_list])
tex_lines.append(head1 + head2 + r" \\")
tex_lines.append(r"\midrule")
for r in rows:
    line = r"{} & {} & {:.1f} & {:.1f}".format(r[0], r[1], r[2], r[3])
    for k in range(len(tau_list)):
        v = r[4+k]
        line += " & " + (fmt(v) if isinstance(v,str) else f"{v:.3f}")
    tex_lines.append(line + r" \\")
tex_lines.append(r"\bottomrule")
tex_lines.append(r"\end{tabular}")
tex_lines.append(r"\end{table}")

with open(out_tex, "w") as f:
    f.write("\n".join(tex_lines))
print(f"[saved] {out_tex}")

# Also print path summary for convenience
print("\n[Artifacts]")
print(" -", out_pdf)
print(" -", out_tex)


[base] n=11576  mean=119.0  p50=90.0
[Proposed (CGTAD)] n=36167  mean=59.0  p50=50.5
[Proposed (matchN)] n=11576  mean=58.9  p50=50.5
[saved] /content/tdi_dice_sensitivity.pdf
[saved] /content/table_sota_tdi_dice.tex

[Artifacts]
 - /content/tdi_dice_sensitivity.pdf
 - /content/table_sota_tdi_dice.tex


In [35]:
# === Generate SOTA candidate: CSD-Prob (DIPY) → /content/csd_prob.trk ===
import numpy as np, nibabel as nib
from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel, auto_response_ssst
from dipy.direction import ProbabilisticDirectionGetter
from dipy.data import default_sphere
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.stopping_criterion import BinaryStoppingCriterion
from dipy.tracking.utils import seeds_from_mask
from dipy.io.stateful_tractogram import StatefulTractogram, Space
from dipy.io.streamline import save_trk

# 1) response（FAしきいは安全側）
try:
    response, _ = auto_response_ssst(gtab, data, roi_radius=10, fa_thr=0.6)
except Exception:
    # Fallback: DTI中央値レスポンス
    from dipy.reconst.dti import TensorModel
    ten = TensorModel(gtab).fit(data, mask=mask)
    evals = np.clip(ten.evals, 1e-4, 3e-3)
    S0 = float(np.median(data[mask].mean(axis=-1)))
    response = (np.median(evals[mask], axis=0), S0)

# 2) CSD fit
csd = ConstrainedSphericalDeconvModel(gtab, response, sh_order=8)
csd_fit = csd.fit(data, mask=mask)

# 3) Probabilistic direction getter（max_angleは30°前後が一般的）
pdg = ProbabilisticDirectionGetter.from_shcoeff(
    csd_fit.shm_coeff, max_angle=30., sphere=default_sphere
)

# 4) seeds / stopping
stop  = BinaryStoppingCriterion(mask)
seeds = seeds_from_mask(mask, density=1, affine=affine)

# 5) tracking（mm座標で出力）
streamlines = list(LocalTracking(pdg, stop, seeds, affine=affine, step_size=0.5, return_all=False))

# 6) save
ref_img = nib.Nifti1Image(np.zeros(data.shape[:3], dtype=np.float32), affine)
sft = StatefulTractogram(streamlines, ref_img, Space.RASMM)
save_trk(sft, "/content/csd_prob.trk", bbox_valid_check=False)
print("[saved] /content/csd_prob.trk  n=", len(streamlines))


100%|██████████| 184907/184907 [01:17<00:00, 2380.30it/s]


[saved] /content/csd_prob.trk  n= 58208


In [36]:
# === Convert external SOTA outputs to .trk (unified) ===
import os, numpy as np, nibabel as nib, pathlib
from dipy.io.stateful_tractogram import StatefulTractogram, Space
from dipy.io.streamline import save_trk

ref_img = nib.Nifti1Image(np.zeros(data.shape[:3], dtype=np.float32), affine)

def convert_to_trk(in_path, out_path=None):
    p = pathlib.Path(in_path)
    ext = p.suffix.lower()
    if out_path is None:
        out_path = str(p.with_suffix(".trk"))
    sl = None

    if ext == ".trk":
        print(f"[skip] already .trk: {p}")
        return str(p)

    elif ext == ".tck":
        # MRtrix iFOD2 等
        from nibabel.streamlines import load as load_sl
        tck = load_sl(str(p))
        sl = [np.asarray(s, dtype=np.float32) for s in tck.streamlines]  # MRtrixはRASMM
        sft = StatefulTractogram(sl, ref_img, Space.RASMM)
        save_trk(sft, out_path, bbox_valid_check=False)
        print(f"[conv] TCK→TRK: {p.name} → {out_path}  n={len(sl)}")
        return out_path

    elif ext in [".vtk", ".vtp"]:
        # UKFTractography (VTK PolyData)
        try:
            import pyvista as pv
        except Exception as e:
            raise RuntimeError("pyvista が必要です。`pip install pyvista` を実行してください。") from e
        mesh = pv.read(str(p))
        # PolyLines をstreamlines(list of Nx3)へ
        sl = []
        for cell in mesh.lines.reshape((-1, 3)) if mesh.lines.size % 3 == 0 else []:
            pass  # 古いフォーマット回避のため別経路で読む
        # より堅牢な抽出
        for ids in mesh.extract_all_edges().lines.reshape(-1, 3) if False else []:
            pass
        # 汎用：pyvistaの `mesh.lines` は [npts, id0, id1, ..., npts, ...] 形式の場合あり
        arr = mesh.lines.reshape((-1,))  # 1D
        i = 0
        while i < len(arr):
            n = int(arr[i]); i += 1
            idx = arr[i:i+n]; i += n
            pts = np.asarray(mesh.points[idx], dtype=np.float32)
            sl.append(pts)

        if not sl:  # fallback: poly_data.lines_dict (pyvista>=0.43)
            try:
                for _, idx in mesh.lines_dict.items():
                    pts = np.asarray(mesh.points[idx], dtype=np.float32)
                    sl.append(pts)
            except Exception:
                pass

        if not sl:
            raise RuntimeError(f"VTK 読み出しに失敗: {p}")

        sft = StatefulTractogram(sl, ref_img, Space.RASMM)  # UKF はRASMM
        save_trk(sft, out_path, bbox_valid_check=False)
        print(f"[conv] VTK→TRK: {p.name} → {out_path}  n={len(sl)}")
        return out_path

    else:
        raise ValueError(f"Unsupported extension: {ext}")

# 例：ここに外部SOTAのパス（存在するものだけでOK）
external_inputs = {
    "SOTA: iFOD2": "/content/ifod2.tck",          # MRtrix tck の想定出力
    "SOTA: UKF":   "/content/ukf.vtk",            # UKFTractography の想定出力
    "FSL Probtrackx": "/content/probtrackx.tck",  # tck化されているなら可
}

converted = {}
for name, path in external_inputs.items():
    if os.path.exists(path):
        try:
            out_trk = convert_to_trk(path)
            converted[name] = out_trk
        except Exception as e:
            print(f"[warn] convert failed for {name}: {e}")
    else:
        print(f"[skip] not found: {path}")

print("[converted]", converted)


[skip] not found: /content/ifod2.tck
[skip] not found: /content/ukf.vtk
[skip] not found: /content/probtrackx.tck
[converted] {}


In [37]:
# === Re-run sensitivity & table including SOTA methods ===
import os, numpy as np, nibabel as nib, matplotlib.pyplot as plt
from dipy.io.streamline import load_trk
from dipy.io.stateful_tractogram import Space
from dipy.tracking.streamline import length

# ---- 設定 ----
base_trk = "/content/baseline_Lmin20.trk"
methods = {
    "Proposed (CGTAD)"  : "/content/B_streamlines_Lmin20.trk",
    "Proposed (matchN)" : "/content/B_streamlines_Lmin20_matchN.trk",
    "CSD-Prob (DIPY)"   : "/content/csd_prob.trk",           # セルAの生成物
    # ↓ セルBで変換できたら自動で拾います
    "SOTA: iFOD2"       : "/content/ifod2.trk",
    "SOTA: UKF"         : "/content/ukf.trk",
    "FSL Probtrackx"    : "/content/probtrackx.trk",
}
tau_list = [1, 3, 5, 10, 15, 20]
out_pdf = "/content/tdi_dice_sensitivity_all.pdf"
out_tex = "/content/table_sota_tdi_dice_all.tex"

def load_mm(p):
    sft = load_trk(p, "same", bbox_valid_check=False)
    sft.to_space(Space.RASMM)
    sl = list(sft.streamlines)
    L  = np.array([length(s) for s in sl], dtype=float)
    return sl, L, sft

def load_vox(p):
    sft = load_trk(p, "same", bbox_valid_check=False)
    sft.to_space(Space.VOX)
    return list(sft.streamlines), sft

def tdi_counts(sl_vox, shape):
    c = np.zeros(shape, dtype=np.int32)
    for s in sl_vox:
        ijk = np.rint(s).astype(int)
        valid = (ijk[:,0]>=0)&(ijk[:,1]>=0)&(ijk[:,2]>=0)
        ijk = ijk[valid]
        if len(ijk)==0: continue
        ijk[:,0] = np.clip(ijk[:,0], 0, shape[0]-1)
        ijk[:,1] = np.clip(ijk[:,1], 0, shape[1]-1)
        ijk[:,2] = np.clip(ijk[:,2], 0, shape[2]-1)
        c[ijk[:,0], ijk[:,1], ijk[:,2]] += 1
    return c

dice = lambda A,B: 2*np.logical_and(A,B).sum()/(A.sum()+B.sum()+1e-8)

# ---- Base ----
assert os.path.exists(base_trk), f"not found: {base_trk}"
sl_b_vox, sft_b = load_vox(base_trk)
sl_b_mm,  L_b,  _ = load_mm(base_trk)
shape = sft_b.dimensions[::-1]
T_base = tdi_counts(sl_b_vox, shape)

name2L = {"Baseline": L_b}
curves = {}

# ---- Each method ----
plt.rcParams["pdf.fonttype"]=42; plt.rcParams["ps.fonttype"]=42
plt.figure(figsize=(6,4))
for name, p in methods.items():
    if not os.path.exists(p):
        print(f"[skip] {name}: {p}")
        continue
    sl_v, _ = load_vox(p)
    _, L, _ = load_mm(p)
    name2L[name] = L
    T = tdi_counts(sl_v, shape)
    y = []
    for tau in tau_list:
        y.append(dice(T_base>=tau, T>=tau))
    curves[name] = y
    plt.plot(tau_list, y, marker="o", label=name)
plt.xlabel(r"TDI threshold $\tau$  (counts/voxel)")
plt.ylabel("Dice vs Baseline"); plt.xticks(tau_list); plt.ylim(0,1); plt.grid(True,alpha=.3)
plt.legend(fontsize=8); plt.tight_layout(); plt.savefig(out_pdf); plt.close()
print("[saved]", out_pdf)

# ---- LaTeX table ----
rows = []
rows.append(["Baseline", len(name2L["Baseline"]), float(np.mean(name2L["Baseline"])), float(np.percentile(name2L["Baseline"],50))] + ["-"]*len(tau_list))
for name in methods:
    if name not in name2L: continue
    L = name2L[name]
    stats = [len(L), float(np.mean(L)), float(np.percentile(L,50))]
    dlist = [float(v) for v in curves[name]]
    rows.append([name] + stats + dlist)

def fmt_row(r):
    line = r"{} & {} & {:.1f} & {:.1f}".format(r[0], r[1], r[2], r[3])
    for v in r[4:]:
        line += " & " + (f"{v:.3f}" if isinstance(v,float) else v)
    return line + r" \\"

tex = []
tex += [r"\begin{table*}[t]", r"\centering"]
tex += [r"\caption{TDI--Dice sensitivity vs.\ Baseline after length filtering (all methods).}"]
tex += [r"\label{tab:tdi_dice_sensitivity_all}"]
tex += [r"\setlength{\tabcolsep}{3pt}\renewcommand{\arraystretch}{1.05}"]
tex += [r"\resizebox{\textwidth}{!}{%"]
tex += [r"\begin{tabular}{lrrrr" + "r"*len(tau_list) + r"}", r"\toprule"]
tex += [r"Method & $n$ & mean [mm] & $p_{50}$ [mm] & \multicolumn{" + str(len(tau_list)) + r"}{c}{Dice vs.\ Baseline at $\tau$} \\", r"\cmidrule(l){5-" + str(4+len(tau_list)) + r"}"]
tex += [r" &  &  &  & " + " & ".join([str(t) for t in tau_list]) + r" \\", r"\midrule"]
for r in rows: tex += [fmt_row(r)]
tex += [r"\bottomrule", r"\end{tabular}%", r"}", r"\end{table*}"]

with open(out_tex, "w") as f:
    f.write("\n".join(tex))
print("[saved]", out_tex)


[skip] SOTA: iFOD2: /content/ifod2.trk
[skip] SOTA: UKF: /content/ukf.trk
[skip] FSL Probtrackx: /content/probtrackx.trk
[saved] /content/tdi_dice_sensitivity_all.pdf
[saved] /content/table_sota_tdi_dice_all.tex
